## Initialization

## Load Data

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import random
import numpy as np

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam


## EDA

In [ ]:
df_labels = pd.read_csv('/datasets/faces/labels.csv')

In [ ]:
age_count = df_labels.groupby('real_age').count()
age_count = age_count.reset_index().rename(columns={0: 'real_age_new', 'file_name': 'count'})
age_count_sorted = age_count.sort_values(by='count', ascending=False)
display(age_count_sorted)

age_count_sorted.plot(kind='hist',
                     bins=20, 
                     alpha=0.7,
                     title='Age Distribution in Photos',
                     y='count')

age_count_sorted.plot(kind='scatter',
                     title='Age Distribution in Photos Scatter Plot',
                     x='real_age_new',
                     y='count',
                     xlabel='Age in Photo',
                     ylabel='Number of Photos')
plt.show()

In [ ]:
sample_indices = random.sample(range(len(train_datagen_flow)), 10)


for batch_index in sample_indices:
    data, _ = train_datagen_flow[batch_index] 

    for i in range(len(data)):
        array = data[i] 
        plt.imshow(array)
        plt.axis('off')
        plt.show()

## Modelling

In [ ]:
def load_train(path):
    
    """
    It loads the train part of dataset from path
    """
    
    train_datagen = ImageDataGenerator(
        validation_split=0.2,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rotation_range=90,
        zoom_range=0.2,
        rescale=1/255)

    train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe=df_labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=64,
        class_mode='raw',
        subset='training',
        seed=12345,
        )
    return train_datagen_flow

In [ ]:
def load_test(path):
    
    """
    It loads the validation/test part of dataset from path
    """
    
    test_datagen = ImageDataGenerator(validation_split=0.2, rescale=1/255)
    
    test_datagen_flow = test_datagen.flow_from_dataframe(
    dataframe=df_labels,
    directory='/datasets/faces/final_files/',
    x_col='file_name',
    y_col='real_age',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    subset='validation',
    seed=12345,
    )

    return test_datagen_flow

In [ ]:
def create_model(input_shape):
    
    """
    It defines the model
    """
   
    optimizer = Adam(lr=0.00005)


    backbone = ResNet50(
        input_shape=input_shape, weights='imagenet', include_top=False
    )

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())

    model.add(Dense(1, activation='relu'))



    model.compile(

        optimizer=optimizer,

        loss='mse',

        metrics=['mae']
    

    )
    return model


In [ ]:
def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):

    """
    Trains the model given the parameters
    """

    model.fit(train_data,
          validation_data=test_data,
          batch_size=batch_size,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch,
          validation_steps=validation_steps,
          verbose=1) 


    return model

### Prepare the Script to Run on the GPU Platform

In [ ]:
# prepare a script to run on the GPU platform

init_str = 'Get started'


import inspect

with open('run_model_on_gpu.py', 'w') as f:
    
    f.write(init_str)
    f.write('\n\n')
        
    for fn_name in [load_train, load_test, create_model, train_model]:
        
        src = inspect.getsource(fn_name)
        f.write(src)
        f.write('\n\n')

### Output

Train for 95 steps, validate for 24 steps
Epoch 1/20
2023-10-19 19:12:19.254703: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-10-19 19:12:20.268845: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7

 1/95 [..............................] - ETA: 18:53 - loss: 962.2367 - mae: 25.9471
 2/95 [..............................] - ETA: 9:31 - loss: 1116.5602 - mae: 28.6662
 3/95 [..............................] - ETA: 6:23 - loss: 1042.0891 - mae: 28.0872
 4/95 [>.............................] - ETA: 4:48 - loss: 1141.2063 - mae: 29.3970
 5/95 [>.............................] - ETA: 3:52 - loss: 1126.1599 - mae: 29.1570
 6/95 [>.............................] - ETA: 3:14 - loss: 1172.7910 - mae: 29.6985
 7/95 [=>............................] - ETA: 2:53 - loss: 1162.1312 - mae: 29.4458
 8/95 [=>............................] - ETA: 2:41 - loss: 1169.1107 - mae: 29.4408
 9/95 [=>............................] - ETA: 2:32 - loss: 1151.5128 - mae: 29.3568
10/95 [==>...........................] - ETA: 2:24 - loss: 1151.0072 - mae: 29.4243
11/95 [==>...........................] - ETA: 2:18 - loss: 1146.5202 - mae: 29.3266
12/95 [==>...........................] - ETA: 2:12 - loss: 1167.8424 - mae: 29.6409
13/95 [===>..........................] - ETA: 2:07 - loss: 1165.6939 - mae: 29.6603
14/95 [===>..........................] - ETA: 2:02 - loss: 1170.4573 - mae: 29.7880
15/95 [===>..........................] - ETA: 1:59 - loss: 1157.1207 - mae: 29.5984
16/95 [====>.........................] - ETA: 1:56 - loss: 1150.0128 - mae: 29.4614
17/95 [====>.........................] - ETA: 1:52 - loss: 1131.6972 - mae: 29.2426
18/95 [====>.........................] - ETA: 1:49 - loss: 1128.1284 - mae: 29.2018
19/95 [=====>........................] - ETA: 1:46 - loss: 1115.6728 - mae: 29.0603
20/95 [=====>........................] - ETA: 1:43 - loss: 1093.4661 - mae: 28.6960
21/95 [=====>........................] - ETA: 1:41 - loss: 1076.7139 - mae: 28.4008
22/95 [=====>........................] - ETA: 1:40 - loss: 1070.7271 - mae: 28.3234
23/95 [======>.......................] - ETA: 1:38 - loss: 1063.0275 - mae: 28.1611
24/95 [======>.......................] - ETA: 1:36 - loss: 1042.2605 - mae: 27.8760
25/95 [======>.......................] - ETA: 1:33 - loss: 1023.8200 - mae: 27.5616
26/95 [=======>......................] - ETA: 1:31 - loss: 1010.1835 - mae: 27.3193
27/95 [=======>......................] - ETA: 1:29 - loss: 993.0825 - mae: 27.0205 
28/95 [=======>......................] - ETA: 1:27 - loss: 976.6466 - mae: 26.7806
29/95 [========>.....................] - ETA: 1:26 - loss: 963.9215 - mae: 26.5617
30/95 [========>.....................] - ETA: 1:24 - loss: 948.5474 - mae: 26.2997
31/95 [========>.....................] - ETA: 1:23 - loss: 936.1580 - mae: 26.1010
32/95 [=========>....................] - ETA: 1:21 - loss: 927.5209 - mae: 25.9146
33/95 [=========>....................] - ETA: 1:19 - loss: 915.1779 - mae: 25.7100
34/95 [=========>....................] - ETA: 1:17 - loss: 904.7384 - mae: 25.5219
35/95 [==========>...................] - ETA: 1:17 - loss: 892.3477 - mae: 25.3227
36/95 [==========>...................] - ETA: 1:14 - loss: 879.7031 - mae: 25.0990
37/95 [==========>...................] - ETA: 1:12 - loss: 869.4645 - mae: 24.9297
38/95 [===========>..................] - ETA: 1:11 - loss: 861.8938 - mae: 24.8038
39/95 [===========>..................] - ETA: 1:10 - loss: 852.2809 - mae: 24.6439
40/95 [===========>..................] - ETA: 1:08 - loss: 844.8932 - mae: 24.4965
41/95 [===========>..................] - ETA: 1:07 - loss: 833.1495 - mae: 24.2763
42/95 [============>.................] - ETA: 1:06 - loss: 823.1225 - mae: 24.1093
43/95 [============>.................] - ETA: 1:04 - loss: 812.3737 - mae: 23.8985
44/95 [============>.................] - ETA: 1:03 - loss: 803.3039 - mae: 23.7261
45/95 [=============>................] - ETA: 1:01 - loss: 794.5555 - mae: 23.5635
46/95 [=============>................] - ETA: 1:00 - loss: 787.4501 - mae: 23.4188
47/95 [=============>................] - ETA: 58s - loss: 776.7083 - mae: 23.2124 
48/95 [==============>...............] - ETA: 57s - loss: 766.7595 - mae: 23.0124
49/95 [==============>...............] - ETA: 56s - loss: 757.4569 - mae: 22.8220
50/95 [==============>...............] - ETA: 54s - loss: 748.2341 - mae: 22.6532
51/95 [===============>..............] - ETA: 53s - loss: 738.5754 - mae: 22.4563
52/95 [===============>..............] - ETA: 52s - loss: 730.7642 - mae: 22.3090
53/95 [===============>..............] - ETA: 50s - loss: 720.9336 - mae: 22.1004
54/95 [================>.............] - ETA: 49s - loss: 712.7935 - mae: 21.9214
55/95 [================>.............] - ETA: 48s - loss: 704.7550 - mae: 21.7440
56/95 [================>.............] - ETA: 46s - loss: 696.1881 - mae: 21.5696
57/95 [=================>............] - ETA: 45s - loss: 688.0588 - mae: 21.3908
58/95 [=================>............] - ETA: 44s - loss: 679.6099 - mae: 21.2127
59/95 [=================>............] - ETA: 43s - loss: 672.6735 - mae: 21.0732
60/95 [=================>............] - ETA: 42s - loss: 665.4617 - mae: 20.9149
61/95 [==================>...........] - ETA: 40s - loss: 657.8762 - mae: 20.7606
62/95 [==================>...........] - ETA: 39s - loss: 651.6109 - mae: 20.6286
63/95 [==================>...........] - ETA: 38s - loss: 646.1104 - mae: 20.5161
64/95 [===================>..........] - ETA: 36s - loss: 639.3443 - mae: 20.3739
65/95 [===================>..........] - ETA: 35s - loss: 633.3527 - mae: 20.2618
66/95 [===================>..........] - ETA: 34s - loss: 628.4664 - mae: 20.1645
67/95 [====================>.........] - ETA: 33s - loss: 623.1863 - mae: 20.0521
68/95 [====================>.........] - ETA: 32s - loss: 617.2644 - mae: 19.9218
69/95 [====================>.........] - ETA: 30s - loss: 611.0248 - mae: 19.7878
70/95 [=====================>........] - ETA: 29s - loss: 604.8355 - mae: 19.6518
71/95 [=====================>........] - ETA: 28s - loss: 598.4107 - mae: 19.5080
72/95 [=====================>........] - ETA: 27s - loss: 594.3127 - mae: 19.4240
73/95 [======================>.......] - ETA: 25s - loss: 589.9045 - mae: 19.3465
74/95 [======================>.......] - ETA: 24s - loss: 584.6745 - mae: 19.2421
75/95 [======================>.......] - ETA: 23s - loss: 579.8215 - mae: 19.1270
76/95 [=======================>......] - ETA: 22s - loss: 574.5600 - mae: 19.0225
77/95 [=======================>......] - ETA: 21s - loss: 570.2116 - mae: 18.9367
78/95 [=======================>......] - ETA: 19s - loss: 565.4221 - mae: 18.8227
79/95 [=======================>......] - ETA: 18s - loss: 561.0829 - mae: 18.7145
80/95 [========================>.....] - ETA: 17s - loss: 557.0460 - mae: 18.6310
81/95 [========================>.....] - ETA: 16s - loss: 552.4691 - mae: 18.5250
82/95 [========================>.....] - ETA: 15s - loss: 547.8775 - mae: 18.4236
83/95 [=========================>....] - ETA: 13s - loss: 542.6739 - mae: 18.3070
84/95 [=========================>....] - ETA: 12s - loss: 538.8872 - mae: 18.2269
85/95 [=========================>....] - ETA: 11s - loss: 533.5214 - mae: 18.0931
86/95 [==========================>...] - ETA: 10s - loss: 529.0408 - mae: 17.9972
87/95 [==========================>...] - ETA: 9s - loss: 525.1097 - mae: 17.9054 
88/95 [==========================>...] - ETA: 8s - loss: 520.6974 - mae: 17.8064
89/95 [===========================>..] - ETA: 6s - loss: 516.3175 - mae: 17.7023
90/95 [===========================>..] - ETA: 5s - loss: 513.0126 - mae: 17.6208
91/95 [===========================>..] - ETA: 4s - loss: 508.9112 - mae: 17.5283
92/95 [============================>.] - ETA: 3s - loss: 505.5359 - mae: 17.4577
93/95 [============================>.] - ETA: 2s - loss: 503.1281 - mae: 17.3993
94/95 [============================>.] - ETA: 1s - loss: 499.2762 - mae: 17.3138
95/95 [==============================] - 121s 1s/step - loss: 495.7238 - mae: 17.2343 - val_loss: 1063.6291 - val_mae: 27.9191
Epoch 2/20

 1/95 [..............................] - ETA: 1:57 - loss: 179.6504 - mae: 10.4078
 2/95 [..............................] - ETA: 1:43 - loss: 154.3787 - mae: 9.7996 
 3/95 [..............................] - ETA: 1:35 - loss: 170.5615 - mae: 10.2348
 4/95 [>.............................] - ETA: 1:31 - loss: 160.6176 - mae: 9.9070 
 5/95 [>.............................] - ETA: 1:27 - loss: 168.3955 - mae: 9.7988
 6/95 [>.............................] - ETA: 1:26 - loss: 165.0132 - mae: 9.7956
 7/95 [=>............................] - ETA: 1:25 - loss: 157.9360 - mae: 9.5718
 8/95 [=>............................] - ETA: 1:23 - loss: 153.3712 - mae: 9.4684
 9/95 [=>............................] - ETA: 1:22 - loss: 155.7319 - mae: 9.5141
10/95 [==>...........................] - ETA: 1:21 - loss: 158.0730 - mae: 9.5572
11/95 [==>...........................] - ETA: 1:20 - loss: 158.3924 - mae: 9.5382
12/95 [==>...........................] - ETA: 1:20 - loss: 153.3846 - mae: 9.3544
13/95 [===>..........................] - ETA: 1:18 - loss: 155.7310 - mae: 9.4355
14/95 [===>..........................] - ETA: 1:17 - loss: 155.4720 - mae: 9.4533
15/95 [===>..........................] - ETA: 1:16 - loss: 153.9168 - mae: 9.4132
16/95 [====>.........................] - ETA: 1:15 - loss: 155.2669 - mae: 9.4503
17/95 [====>.........................] - ETA: 1:14 - loss: 156.1811 - mae: 9.4887
18/95 [====>.........................] - ETA: 1:13 - loss: 154.2082 - mae: 9.4347
19/95 [=====>........................] - ETA: 1:13 - loss: 154.9756 - mae: 9.4374
20/95 [=====>........................] - ETA: 1:12 - loss: 152.8159 - mae: 9.3727
21/95 [=====>........................] - ETA: 1:11 - loss: 150.2952 - mae: 9.2999
22/95 [=====>........................] - ETA: 1:10 - loss: 149.3123 - mae: 9.2578
23/95 [======>.......................] - ETA: 1:09 - loss: 146.5527 - mae: 9.1508
24/95 [======>.......................] - ETA: 1:08 - loss: 145.2827 - mae: 9.1270
25/95 [======>.......................] - ETA: 1:07 - loss: 145.5274 - mae: 9.1394
26/95 [=======>......................] - ETA: 1:06 - loss: 144.8883 - mae: 9.1381
27/95 [=======>......................] - ETA: 1:05 - loss: 144.2381 - mae: 9.1174
28/95 [=======>......................] - ETA: 1:04 - loss: 146.1923 - mae: 9.1554
29/95 [========>.....................] - ETA: 1:03 - loss: 144.7683 - mae: 9.1141
30/95 [========>.....................] - ETA: 1:02 - loss: 144.5592 - mae: 9.1315
31/95 [========>.....................] - ETA: 1:01 - loss: 143.0809 - mae: 9.0782
32/95 [=========>....................] - ETA: 1:01 - loss: 143.0225 - mae: 9.0963
33/95 [=========>....................] - ETA: 1:00 - loss: 141.9440 - mae: 9.0774
34/95 [=========>....................] - ETA: 59s - loss: 141.2830 - mae: 9.0568 
35/95 [==========>...................] - ETA: 58s - loss: 140.1938 - mae: 9.0114
36/95 [==========>...................] - ETA: 57s - loss: 140.2670 - mae: 9.0154
37/95 [==========>...................] - ETA: 56s - loss: 139.5394 - mae: 8.9848
38/95 [===========>..................] - ETA: 55s - loss: 139.9836 - mae: 8.9945
39/95 [===========>..................] - ETA: 54s - loss: 139.0540 - mae: 8.9748
40/95 [===========>..................] - ETA: 53s - loss: 139.5589 - mae: 8.9891
41/95 [===========>..................] - ETA: 52s - loss: 139.5104 - mae: 8.9951
42/95 [============>.................] - ETA: 51s - loss: 138.9518 - mae: 8.9735
43/95 [============>.................] - ETA: 50s - loss: 138.8600 - mae: 8.9732
44/95 [============>.................] - ETA: 49s - loss: 139.0352 - mae: 8.9687
45/95 [=============>................] - ETA: 48s - loss: 139.8121 - mae: 8.9812
46/95 [=============>................] - ETA: 47s - loss: 139.0114 - mae: 8.9533
47/95 [=============>................] - ETA: 46s - loss: 138.0780 - mae: 8.9295
48/95 [==============>...............] - ETA: 45s - loss: 137.7075 - mae: 8.9179
49/95 [==============>...............] - ETA: 44s - loss: 137.5152 - mae: 8.9128
50/95 [==============>...............] - ETA: 43s - loss: 137.1922 - mae: 8.9179
51/95 [===============>..............] - ETA: 42s - loss: 137.1705 - mae: 8.9198
52/95 [===============>..............] - ETA: 41s - loss: 137.5185 - mae: 8.9249
53/95 [===============>..............] - ETA: 40s - loss: 137.0509 - mae: 8.9176
54/95 [================>.............] - ETA: 39s - loss: 137.2701 - mae: 8.9135
55/95 [================>.............] - ETA: 38s - loss: 136.2927 - mae: 8.8834
56/95 [================>.............] - ETA: 37s - loss: 136.2097 - mae: 8.8821
57/95 [=================>............] - ETA: 36s - loss: 137.2512 - mae: 8.9198
58/95 [=================>............] - ETA: 35s - loss: 137.7373 - mae: 8.9413
59/95 [=================>............] - ETA: 34s - loss: 136.9866 - mae: 8.9266
60/95 [=================>............] - ETA: 33s - loss: 137.3763 - mae: 8.9423
61/95 [==================>...........] - ETA: 32s - loss: 137.6736 - mae: 8.9594
62/95 [==================>...........] - ETA: 31s - loss: 138.0739 - mae: 8.9641
63/95 [==================>...........] - ETA: 30s - loss: 138.4878 - mae: 8.9692
64/95 [===================>..........] - ETA: 29s - loss: 138.0123 - mae: 8.9590
65/95 [===================>..........] - ETA: 28s - loss: 138.8505 - mae: 8.9790
66/95 [===================>..........] - ETA: 27s - loss: 139.6000 - mae: 8.9969
67/95 [====================>.........] - ETA: 26s - loss: 139.7556 - mae: 9.0008
68/95 [====================>.........] - ETA: 25s - loss: 139.7911 - mae: 9.0014
69/95 [====================>.........] - ETA: 24s - loss: 139.4233 - mae: 8.9869
70/95 [=====================>........] - ETA: 23s - loss: 139.0385 - mae: 8.9763
71/95 [=====================>........] - ETA: 22s - loss: 139.3366 - mae: 8.9766
72/95 [=====================>........] - ETA: 22s - loss: 139.7018 - mae: 8.9894
73/95 [======================>.......] - ETA: 21s - loss: 139.4361 - mae: 8.9850
74/95 [======================>.......] - ETA: 20s - loss: 139.2273 - mae: 8.9883
75/95 [======================>.......] - ETA: 19s - loss: 139.1006 - mae: 8.9831
76/95 [=======================>......] - ETA: 18s - loss: 139.0127 - mae: 8.9738
77/95 [=======================>......] - ETA: 17s - loss: 138.7527 - mae: 8.9694
78/95 [=======================>......] - ETA: 16s - loss: 138.0981 - mae: 8.9410
79/95 [=======================>......] - ETA: 15s - loss: 137.8484 - mae: 8.9360
80/95 [========================>.....] - ETA: 14s - loss: 137.9033 - mae: 8.9436
81/95 [========================>.....] - ETA: 13s - loss: 137.8040 - mae: 8.9466
82/95 [========================>.....] - ETA: 12s - loss: 137.8924 - mae: 8.9555
83/95 [=========================>....] - ETA: 11s - loss: 137.3532 - mae: 8.9368
84/95 [=========================>....] - ETA: 10s - loss: 137.4252 - mae: 8.9400
85/95 [=========================>....] - ETA: 9s - loss: 137.0829 - mae: 8.9361 
86/95 [==========================>...] - ETA: 8s - loss: 136.9961 - mae: 8.9329
87/95 [==========================>...] - ETA: 7s - loss: 136.7768 - mae: 8.9177
88/95 [==========================>...] - ETA: 6s - loss: 136.5063 - mae: 8.9000
89/95 [===========================>..] - ETA: 5s - loss: 136.2930 - mae: 8.8963
90/95 [===========================>..] - ETA: 4s - loss: 135.8746 - mae: 8.8842
91/95 [===========================>..] - ETA: 3s - loss: 135.5278 - mae: 8.8705
92/95 [============================>.] - ETA: 2s - loss: 135.4996 - mae: 8.8740
93/95 [============================>.] - ETA: 1s - loss: 135.5070 - mae: 8.8678
94/95 [============================>.] - ETA: 0s - loss: 135.5338 - mae: 8.8747
95/95 [==============================] - 99s 1s/step - loss: 135.7794 - mae: 8.8857 - val_loss: 1031.4883 - val_mae: 27.3650
Epoch 3/20

 1/95 [..............................] - ETA: 1:46 - loss: 138.2851 - mae: 8.7266
 2/95 [..............................] - ETA: 1:34 - loss: 122.9857 - mae: 8.4255
 3/95 [..............................] - ETA: 1:31 - loss: 117.4524 - mae: 8.4746
 4/95 [>.............................] - ETA: 1:30 - loss: 122.1538 - mae: 8.6039
 5/95 [>.............................] - ETA: 1:28 - loss: 118.7916 - mae: 8.5140
 6/95 [>.............................] - ETA: 1:27 - loss: 118.4335 - mae: 8.5570
 7/95 [=>............................] - ETA: 1:25 - loss: 111.0704 - mae: 8.1985
 8/95 [=>............................] - ETA: 1:23 - loss: 109.3668 - mae: 8.1199
 9/95 [=>............................] - ETA: 1:23 - loss: 110.1746 - mae: 8.1105
10/95 [==>...........................] - ETA: 1:21 - loss: 114.4591 - mae: 8.2993
11/95 [==>...........................] - ETA: 1:21 - loss: 113.4568 - mae: 8.2456
12/95 [==>...........................] - ETA: 1:19 - loss: 115.3190 - mae: 8.2935
13/95 [===>..........................] - ETA: 1:18 - loss: 119.3045 - mae: 8.3621
14/95 [===>..........................] - ETA: 1:17 - loss: 121.5478 - mae: 8.3623
15/95 [===>..........................] - ETA: 1:16 - loss: 118.8162 - mae: 8.2829
16/95 [====>.........................] - ETA: 1:16 - loss: 121.2693 - mae: 8.3268
17/95 [====>.........................] - ETA: 1:15 - loss: 122.8377 - mae: 8.3925
18/95 [====>.........................] - ETA: 1:14 - loss: 122.1322 - mae: 8.3726
19/95 [=====>........................] - ETA: 1:12 - loss: 121.5243 - mae: 8.3766
20/95 [=====>........................] - ETA: 1:13 - loss: 123.5661 - mae: 8.4622
21/95 [=====>........................] - ETA: 1:12 - loss: 121.9817 - mae: 8.4091
22/95 [=====>........................] - ETA: 1:11 - loss: 123.9637 - mae: 8.4856
23/95 [======>.......................] - ETA: 1:10 - loss: 122.7593 - mae: 8.4652
24/95 [======>.......................] - ETA: 1:09 - loss: 123.1855 - mae: 8.4790
25/95 [======>.......................] - ETA: 1:08 - loss: 120.7996 - mae: 8.4033
26/95 [=======>......................] - ETA: 1:07 - loss: 119.1475 - mae: 8.3572
27/95 [=======>......................] - ETA: 1:06 - loss: 117.8586 - mae: 8.3134
28/95 [=======>......................] - ETA: 1:05 - loss: 117.3784 - mae: 8.3140
29/95 [========>.....................] - ETA: 1:04 - loss: 116.1168 - mae: 8.2765
30/95 [========>.....................] - ETA: 1:03 - loss: 114.4932 - mae: 8.2227
31/95 [========>.....................] - ETA: 1:02 - loss: 114.4259 - mae: 8.2132
32/95 [=========>....................] - ETA: 1:01 - loss: 114.0538 - mae: 8.2099
33/95 [=========>....................] - ETA: 1:00 - loss: 114.9757 - mae: 8.2394
34/95 [=========>....................] - ETA: 59s - loss: 114.0301 - mae: 8.2187 
35/95 [==========>...................] - ETA: 58s - loss: 113.9007 - mae: 8.2249
36/95 [==========>...................] - ETA: 57s - loss: 113.4339 - mae: 8.2065
37/95 [==========>...................] - ETA: 55s - loss: 112.8752 - mae: 8.1715
38/95 [===========>..................] - ETA: 54s - loss: 113.3106 - mae: 8.1725
39/95 [===========>..................] - ETA: 53s - loss: 112.1616 - mae: 8.1262
40/95 [===========>..................] - ETA: 52s - loss: 112.0657 - mae: 8.1210
41/95 [===========>..................] - ETA: 52s - loss: 112.9022 - mae: 8.1454
42/95 [============>.................] - ETA: 51s - loss: 112.9359 - mae: 8.1434
43/95 [============>.................] - ETA: 49s - loss: 113.7422 - mae: 8.1597
44/95 [============>.................] - ETA: 49s - loss: 114.6101 - mae: 8.1910
45/95 [=============>................] - ETA: 48s - loss: 114.6923 - mae: 8.1902
46/95 [=============>................] - ETA: 47s - loss: 114.3966 - mae: 8.1891
47/95 [=============>................] - ETA: 46s - loss: 113.8527 - mae: 8.1633
48/95 [==============>...............] - ETA: 45s - loss: 113.9498 - mae: 8.1782
49/95 [==============>...............] - ETA: 44s - loss: 113.3802 - mae: 8.1598
50/95 [==============>...............] - ETA: 43s - loss: 112.9222 - mae: 8.1412
51/95 [===============>..............] - ETA: 42s - loss: 112.7504 - mae: 8.1301
52/95 [===============>..............] - ETA: 41s - loss: 112.3945 - mae: 8.1232
53/95 [===============>..............] - ETA: 40s - loss: 111.9575 - mae: 8.1020
54/95 [================>.............] - ETA: 39s - loss: 112.2525 - mae: 8.1079
55/95 [================>.............] - ETA: 38s - loss: 112.8562 - mae: 8.1334
56/95 [================>.............] - ETA: 37s - loss: 112.8757 - mae: 8.1258
57/95 [=================>............] - ETA: 36s - loss: 112.3384 - mae: 8.0952
58/95 [=================>............] - ETA: 35s - loss: 111.8663 - mae: 8.0718
59/95 [=================>............] - ETA: 34s - loss: 112.1669 - mae: 8.0921
60/95 [=================>............] - ETA: 33s - loss: 111.8257 - mae: 8.0863
61/95 [==================>...........] - ETA: 32s - loss: 111.8173 - mae: 8.0834
62/95 [==================>...........] - ETA: 31s - loss: 112.5351 - mae: 8.0914
63/95 [==================>...........] - ETA: 30s - loss: 112.4747 - mae: 8.1023
64/95 [===================>..........] - ETA: 29s - loss: 112.1764 - mae: 8.0907
65/95 [===================>..........] - ETA: 28s - loss: 112.4572 - mae: 8.0969
66/95 [===================>..........] - ETA: 27s - loss: 113.0058 - mae: 8.0989
67/95 [====================>.........] - ETA: 26s - loss: 112.4780 - mae: 8.0742
68/95 [====================>.........] - ETA: 25s - loss: 112.3605 - mae: 8.0747
69/95 [====================>.........] - ETA: 24s - loss: 112.1430 - mae: 8.0755
70/95 [=====================>........] - ETA: 23s - loss: 112.0598 - mae: 8.0771
71/95 [=====================>........] - ETA: 22s - loss: 111.6015 - mae: 8.0551
72/95 [=====================>........] - ETA: 21s - loss: 111.2435 - mae: 8.0446
73/95 [======================>.......] - ETA: 20s - loss: 111.6074 - mae: 8.0578
74/95 [======================>.......] - ETA: 20s - loss: 111.8450 - mae: 8.0647
75/95 [======================>.......] - ETA: 19s - loss: 112.0353 - mae: 8.0689
76/95 [=======================>......] - ETA: 18s - loss: 111.9895 - mae: 8.0684
77/95 [=======================>......] - ETA: 17s - loss: 112.0938 - mae: 8.0673
78/95 [=======================>......] - ETA: 16s - loss: 112.7149 - mae: 8.0664
79/95 [=======================>......] - ETA: 15s - loss: 113.6282 - mae: 8.0790
80/95 [========================>.....] - ETA: 14s - loss: 113.4592 - mae: 8.0743
81/95 [========================>.....] - ETA: 13s - loss: 113.4121 - mae: 8.0789
82/95 [========================>.....] - ETA: 12s - loss: 113.7537 - mae: 8.0892
83/95 [=========================>....] - ETA: 11s - loss: 113.9507 - mae: 8.0953
84/95 [=========================>....] - ETA: 10s - loss: 113.6876 - mae: 8.0879
85/95 [=========================>....] - ETA: 9s - loss: 113.7606 - mae: 8.0888 
86/95 [==========================>...] - ETA: 8s - loss: 113.7609 - mae: 8.0873
87/95 [==========================>...] - ETA: 7s - loss: 113.5030 - mae: 8.0787
88/95 [==========================>...] - ETA: 6s - loss: 113.1338 - mae: 8.0634
89/95 [===========================>..] - ETA: 5s - loss: 113.0886 - mae: 8.0616
90/95 [===========================>..] - ETA: 4s - loss: 113.1716 - mae: 8.0651
91/95 [===========================>..] - ETA: 3s - loss: 113.3420 - mae: 8.0662
92/95 [============================>.] - ETA: 2s - loss: 113.1886 - mae: 8.0624
93/95 [============================>.] - ETA: 1s - loss: 113.0619 - mae: 8.0556
94/95 [============================>.] - ETA: 0s - loss: 112.7686 - mae: 8.0498
95/95 [==============================] - 98s 1s/step - loss: 113.1370 - mae: 8.0568 - val_loss: 1032.6904 - val_mae: 27.3819
Epoch 4/20

 1/95 [..............................] - ETA: 1:55 - loss: 72.0367 - mae: 6.1008
 2/95 [..............................] - ETA: 1:39 - loss: 114.3487 - mae: 7.6995
 3/95 [..............................] - ETA: 1:32 - loss: 114.8951 - mae: 7.7247
 4/95 [>.............................] - ETA: 1:36 - loss: 114.5699 - mae: 7.8997
 5/95 [>.............................] - ETA: 1:32 - loss: 103.5651 - mae: 7.5035
 6/95 [>.............................] - ETA: 1:29 - loss: 100.9558 - mae: 7.4813
 7/95 [=>............................] - ETA: 1:27 - loss: 96.1601 - mae: 7.3256 
 8/95 [=>............................] - ETA: 1:25 - loss: 97.7906 - mae: 7.3858
 9/95 [=>............................] - ETA: 1:24 - loss: 99.1582 - mae: 7.4746
10/95 [==>...........................] - ETA: 1:24 - loss: 102.9721 - mae: 7.5951
11/95 [==>...........................] - ETA: 1:23 - loss: 107.0092 - mae: 7.7567
12/95 [==>...........................] - ETA: 1:22 - loss: 104.4597 - mae: 7.6609
13/95 [===>..........................] - ETA: 1:20 - loss: 104.0122 - mae: 7.6455
14/95 [===>..........................] - ETA: 1:19 - loss: 102.2847 - mae: 7.5853
15/95 [===>..........................] - ETA: 1:17 - loss: 102.5001 - mae: 7.6256
16/95 [====>.........................] - ETA: 1:16 - loss: 101.6053 - mae: 7.6064
17/95 [====>.........................] - ETA: 1:14 - loss: 100.6673 - mae: 7.6158
18/95 [====>.........................] - ETA: 1:13 - loss: 100.1591 - mae: 7.6093
19/95 [=====>........................] - ETA: 1:13 - loss: 103.1694 - mae: 7.6814
20/95 [=====>........................] - ETA: 1:12 - loss: 101.7661 - mae: 7.6389
21/95 [=====>........................] - ETA: 1:11 - loss: 100.9269 - mae: 7.5905
22/95 [=====>........................] - ETA: 1:10 - loss: 101.4515 - mae: 7.6246
23/95 [======>.......................] - ETA: 1:09 - loss: 100.9305 - mae: 7.6297
24/95 [======>.......................] - ETA: 1:08 - loss: 102.9892 - mae: 7.6694
25/95 [======>.......................] - ETA: 1:07 - loss: 103.3074 - mae: 7.6861
26/95 [=======>......................] - ETA: 1:06 - loss: 102.3307 - mae: 7.6492
27/95 [=======>......................] - ETA: 1:05 - loss: 102.2271 - mae: 7.6637
28/95 [=======>......................] - ETA: 1:04 - loss: 102.2328 - mae: 7.6896
29/95 [========>.....................] - ETA: 1:03 - loss: 102.8988 - mae: 7.7099
30/95 [========>.....................] - ETA: 1:02 - loss: 102.9480 - mae: 7.7147
31/95 [========>.....................] - ETA: 1:01 - loss: 104.0440 - mae: 7.7451
32/95 [=========>....................] - ETA: 1:00 - loss: 104.4463 - mae: 7.7739
33/95 [=========>....................] - ETA: 59s - loss: 103.9305 - mae: 7.7677 
34/95 [=========>....................] - ETA: 58s - loss: 103.1792 - mae: 7.7312
35/95 [==========>...................] - ETA: 57s - loss: 102.9239 - mae: 7.7288
36/95 [==========>...................] - ETA: 56s - loss: 102.3720 - mae: 7.7171
37/95 [==========>...................] - ETA: 55s - loss: 102.9705 - mae: 7.7433
38/95 [===========>..................] - ETA: 54s - loss: 102.3155 - mae: 7.7217
39/95 [===========>..................] - ETA: 53s - loss: 102.4609 - mae: 7.7358
40/95 [===========>..................] - ETA: 52s - loss: 102.8333 - mae: 7.7407
41/95 [===========>..................] - ETA: 51s - loss: 102.5349 - mae: 7.7335
42/95 [============>.................] - ETA: 50s - loss: 101.9178 - mae: 7.7098
43/95 [============>.................] - ETA: 49s - loss: 101.7196 - mae: 7.6968
44/95 [============>.................] - ETA: 48s - loss: 100.7973 - mae: 7.6573
45/95 [=============>................] - ETA: 47s - loss: 100.5157 - mae: 7.6533
46/95 [=============>................] - ETA: 46s - loss: 99.8692 - mae: 7.6298 
47/95 [=============>................] - ETA: 45s - loss: 99.1203 - mae: 7.6038
48/95 [==============>...............] - ETA: 44s - loss: 98.9994 - mae: 7.5830
49/95 [==============>...............] - ETA: 44s - loss: 98.8621 - mae: 7.5815
50/95 [==============>...............] - ETA: 43s - loss: 98.3232 - mae: 7.5664
51/95 [===============>..............] - ETA: 42s - loss: 98.5670 - mae: 7.5725
52/95 [===============>..............] - ETA: 41s - loss: 98.5108 - mae: 7.5753
53/95 [===============>..............] - ETA: 40s - loss: 98.2253 - mae: 7.5695
54/95 [================>.............] - ETA: 39s - loss: 98.3130 - mae: 7.5729
55/95 [================>.............] - ETA: 38s - loss: 98.3395 - mae: 7.5764
56/95 [================>.............] - ETA: 37s - loss: 98.1398 - mae: 7.5756
57/95 [=================>............] - ETA: 36s - loss: 98.5505 - mae: 7.5828
58/95 [=================>............] - ETA: 35s - loss: 98.1687 - mae: 7.5664
59/95 [=================>............] - ETA: 34s - loss: 97.7863 - mae: 7.5575
60/95 [=================>............] - ETA: 33s - loss: 97.7555 - mae: 7.5515
61/95 [==================>...........] - ETA: 32s - loss: 97.3110 - mae: 7.5327
62/95 [==================>...........] - ETA: 31s - loss: 97.5227 - mae: 7.5416
63/95 [==================>...........] - ETA: 30s - loss: 97.6550 - mae: 7.5614
64/95 [===================>..........] - ETA: 29s - loss: 97.1869 - mae: 7.5464
65/95 [===================>..........] - ETA: 28s - loss: 97.3261 - mae: 7.5489
66/95 [===================>..........] - ETA: 27s - loss: 97.2211 - mae: 7.5483
67/95 [====================>.........] - ETA: 26s - loss: 97.3178 - mae: 7.5501
68/95 [====================>.........] - ETA: 25s - loss: 97.1168 - mae: 7.5431
69/95 [====================>.........] - ETA: 24s - loss: 96.9001 - mae: 7.5407
70/95 [=====================>........] - ETA: 23s - loss: 97.2585 - mae: 7.5572
71/95 [=====================>........] - ETA: 22s - loss: 97.2186 - mae: 7.5564
72/95 [=====================>........] - ETA: 21s - loss: 97.2385 - mae: 7.5564
73/95 [======================>.......] - ETA: 20s - loss: 96.9567 - mae: 7.5471
74/95 [======================>.......] - ETA: 19s - loss: 97.0521 - mae: 7.5495
75/95 [======================>.......] - ETA: 19s - loss: 97.0775 - mae: 7.5416
76/95 [=======================>......] - ETA: 18s - loss: 96.8260 - mae: 7.5399
77/95 [=======================>......] - ETA: 17s - loss: 97.0160 - mae: 7.5511
78/95 [=======================>......] - ETA: 16s - loss: 97.2475 - mae: 7.5601
79/95 [=======================>......] - ETA: 15s - loss: 97.3159 - mae: 7.5580
80/95 [========================>.....] - ETA: 14s - loss: 97.6847 - mae: 7.5657
81/95 [========================>.....] - ETA: 13s - loss: 97.9312 - mae: 7.5693
82/95 [========================>.....] - ETA: 12s - loss: 98.7160 - mae: 7.5865
83/95 [=========================>....] - ETA: 11s - loss: 98.9656 - mae: 7.6054
84/95 [=========================>....] - ETA: 10s - loss: 99.0111 - mae: 7.6091
85/95 [=========================>....] - ETA: 9s - loss: 98.9977 - mae: 7.6013 
86/95 [==========================>...] - ETA: 8s - loss: 99.1336 - mae: 7.6040
87/95 [==========================>...] - ETA: 7s - loss: 98.7313 - mae: 7.5928
88/95 [==========================>...] - ETA: 6s - loss: 98.8711 - mae: 7.5986
89/95 [===========================>..] - ETA: 5s - loss: 99.4040 - mae: 7.6191
90/95 [===========================>..] - ETA: 4s - loss: 99.5656 - mae: 7.6264
91/95 [===========================>..] - ETA: 3s - loss: 99.5132 - mae: 7.6259
92/95 [============================>.] - ETA: 2s - loss: 99.8550 - mae: 7.6327
93/95 [============================>.] - ETA: 1s - loss: 99.7822 - mae: 7.6241
94/95 [============================>.] - ETA: 0s - loss: 100.1669 - mae: 7.6397
95/95 [==============================] - 99s 1s/step - loss: 100.1948 - mae: 7.6401 - val_loss: 1031.9310 - val_mae: 27.3738
Epoch 5/20

 1/95 [..............................] - ETA: 1:45 - loss: 138.2875 - mae: 8.5995
 2/95 [..............................] - ETA: 1:33 - loss: 94.6603 - mae: 7.2083 
 3/95 [..............................] - ETA: 1:30 - loss: 92.2652 - mae: 7.2225
 4/95 [>.............................] - ETA: 1:29 - loss: 87.9972 - mae: 7.1497
 5/95 [>.............................] - ETA: 1:27 - loss: 91.9871 - mae: 7.1135
 6/95 [>.............................] - ETA: 1:26 - loss: 92.0725 - mae: 7.0975
 7/95 [=>............................] - ETA: 1:26 - loss: 93.7854 - mae: 7.1689
 8/95 [=>............................] - ETA: 1:26 - loss: 92.5735 - mae: 7.2022
 9/95 [=>............................] - ETA: 1:24 - loss: 91.8807 - mae: 7.2045
10/95 [==>...........................] - ETA: 1:22 - loss: 90.5226 - mae: 7.1693
11/95 [==>...........................] - ETA: 1:24 - loss: 92.5480 - mae: 7.2639
12/95 [==>...........................] - ETA: 1:22 - loss: 93.9265 - mae: 7.3285
13/95 [===>..........................] - ETA: 1:21 - loss: 91.6108 - mae: 7.2213
14/95 [===>..........................] - ETA: 1:19 - loss: 91.5757 - mae: 7.2608
15/95 [===>..........................] - ETA: 1:19 - loss: 92.5727 - mae: 7.3209
16/95 [====>.........................] - ETA: 1:17 - loss: 91.9974 - mae: 7.3187
17/95 [====>.........................] - ETA: 1:16 - loss: 90.1140 - mae: 7.2591
18/95 [====>.........................] - ETA: 1:15 - loss: 91.3729 - mae: 7.3266
19/95 [=====>........................] - ETA: 1:13 - loss: 88.9979 - mae: 7.2210
20/95 [=====>........................] - ETA: 1:13 - loss: 87.8037 - mae: 7.1810
21/95 [=====>........................] - ETA: 1:12 - loss: 87.4714 - mae: 7.1831
22/95 [=====>........................] - ETA: 1:11 - loss: 87.5383 - mae: 7.1890
23/95 [======>.......................] - ETA: 1:10 - loss: 87.1783 - mae: 7.1843
24/95 [======>.......................] - ETA: 1:09 - loss: 87.1638 - mae: 7.1695
25/95 [======>.......................] - ETA: 1:08 - loss: 87.1790 - mae: 7.2024
26/95 [=======>......................] - ETA: 1:07 - loss: 87.7280 - mae: 7.2058
27/95 [=======>......................] - ETA: 1:06 - loss: 87.3658 - mae: 7.1846
28/95 [=======>......................] - ETA: 1:05 - loss: 87.1482 - mae: 7.1804
29/95 [========>.....................] - ETA: 1:04 - loss: 87.2569 - mae: 7.1841
30/95 [========>.....................] - ETA: 1:03 - loss: 87.1561 - mae: 7.1629
31/95 [========>.....................] - ETA: 1:02 - loss: 86.9715 - mae: 7.1492
32/95 [=========>....................] - ETA: 1:00 - loss: 86.4091 - mae: 7.1411
33/95 [=========>....................] - ETA: 59s - loss: 87.8127 - mae: 7.1697 
34/95 [=========>....................] - ETA: 58s - loss: 88.8726 - mae: 7.1809
35/95 [==========>...................] - ETA: 57s - loss: 89.0487 - mae: 7.1876
36/95 [==========>...................] - ETA: 56s - loss: 88.9789 - mae: 7.1861
37/95 [==========>...................] - ETA: 56s - loss: 89.9429 - mae: 7.2189
38/95 [===========>..................] - ETA: 55s - loss: 90.5887 - mae: 7.2398
39/95 [===========>..................] - ETA: 53s - loss: 91.0451 - mae: 7.2468
40/95 [===========>..................] - ETA: 52s - loss: 91.0656 - mae: 7.2486
41/95 [===========>..................] - ETA: 51s - loss: 90.9776 - mae: 7.2341
42/95 [============>.................] - ETA: 50s - loss: 90.0448 - mae: 7.1927
43/95 [============>.................] - ETA: 49s - loss: 90.0691 - mae: 7.1823
44/95 [============>.................] - ETA: 48s - loss: 91.3853 - mae: 7.2108
45/95 [=============>................] - ETA: 47s - loss: 91.1517 - mae: 7.1968
46/95 [=============>................] - ETA: 46s - loss: 91.0149 - mae: 7.1953
47/95 [=============>................] - ETA: 45s - loss: 90.8252 - mae: 7.1940
48/95 [==============>...............] - ETA: 44s - loss: 90.2756 - mae: 7.1756
49/95 [==============>...............] - ETA: 43s - loss: 90.5142 - mae: 7.1909
50/95 [==============>...............] - ETA: 42s - loss: 90.1863 - mae: 7.1873
51/95 [===============>..............] - ETA: 41s - loss: 89.6960 - mae: 7.1716
52/95 [===============>..............] - ETA: 41s - loss: 89.5325 - mae: 7.1663
53/95 [===============>..............] - ETA: 40s - loss: 89.2466 - mae: 7.1605
54/95 [================>.............] - ETA: 39s - loss: 88.8151 - mae: 7.1478
55/95 [================>.............] - ETA: 38s - loss: 88.2859 - mae: 7.1296
56/95 [================>.............] - ETA: 37s - loss: 88.6079 - mae: 7.1464
57/95 [=================>............] - ETA: 36s - loss: 88.8499 - mae: 7.1600
58/95 [=================>............] - ETA: 35s - loss: 88.9029 - mae: 7.1710
59/95 [=================>............] - ETA: 34s - loss: 88.8014 - mae: 7.1749
60/95 [=================>............] - ETA: 33s - loss: 88.5665 - mae: 7.1612
61/95 [==================>...........] - ETA: 32s - loss: 88.9703 - mae: 7.1718
62/95 [==================>...........] - ETA: 31s - loss: 88.6858 - mae: 7.1516
63/95 [==================>...........] - ETA: 30s - loss: 88.7027 - mae: 7.1394
64/95 [===================>..........] - ETA: 29s - loss: 88.4826 - mae: 7.1319
65/95 [===================>..........] - ETA: 28s - loss: 88.2936 - mae: 7.1222
66/95 [===================>..........] - ETA: 27s - loss: 87.6905 - mae: 7.1026
67/95 [====================>.........] - ETA: 26s - loss: 87.4966 - mae: 7.0933
68/95 [====================>.........] - ETA: 25s - loss: 87.3845 - mae: 7.0978
69/95 [====================>.........] - ETA: 24s - loss: 87.6207 - mae: 7.1066
70/95 [=====================>........] - ETA: 23s - loss: 87.6061 - mae: 7.1013
71/95 [=====================>........] - ETA: 22s - loss: 87.4627 - mae: 7.0986
72/95 [=====================>........] - ETA: 21s - loss: 87.7838 - mae: 7.1165
73/95 [======================>.......] - ETA: 20s - loss: 87.7478 - mae: 7.1154
74/95 [======================>.......] - ETA: 19s - loss: 87.3691 - mae: 7.1001
75/95 [======================>.......] - ETA: 19s - loss: 87.5800 - mae: 7.1087
76/95 [=======================>......] - ETA: 18s - loss: 88.4499 - mae: 7.1349
77/95 [=======================>......] - ETA: 17s - loss: 89.2674 - mae: 7.1589
78/95 [=======================>......] - ETA: 16s - loss: 89.2298 - mae: 7.1670
79/95 [=======================>......] - ETA: 15s - loss: 89.1337 - mae: 7.1699
80/95 [========================>.....] - ETA: 14s - loss: 89.1481 - mae: 7.1753
81/95 [========================>.....] - ETA: 13s - loss: 88.8887 - mae: 7.1648
82/95 [========================>.....] - ETA: 12s - loss: 88.8380 - mae: 7.1604
83/95 [=========================>....] - ETA: 11s - loss: 88.6596 - mae: 7.1568
84/95 [=========================>....] - ETA: 10s - loss: 88.5279 - mae: 7.1506
85/95 [=========================>....] - ETA: 9s - loss: 88.3300 - mae: 7.1426 
86/95 [==========================>...] - ETA: 8s - loss: 88.6432 - mae: 7.1630
87/95 [==========================>...] - ETA: 7s - loss: 88.9644 - mae: 7.1734
88/95 [==========================>...] - ETA: 6s - loss: 89.3090 - mae: 7.1856
89/95 [===========================>..] - ETA: 5s - loss: 89.3429 - mae: 7.1867
90/95 [===========================>..] - ETA: 4s - loss: 88.9908 - mae: 7.1733
91/95 [===========================>..] - ETA: 3s - loss: 89.1292 - mae: 7.1828
92/95 [============================>.] - ETA: 2s - loss: 88.8857 - mae: 7.1741
93/95 [============================>.] - ETA: 1s - loss: 88.6389 - mae: 7.1657
94/95 [============================>.] - ETA: 0s - loss: 88.5309 - mae: 7.1625
95/95 [==============================] - 99s 1s/step - loss: 88.6115 - mae: 7.1596 - val_loss: 801.3504 - val_mae: 23.2964
Epoch 6/20

 1/95 [..............................] - ETA: 2:22 - loss: 73.3696 - mae: 7.0265
 2/95 [..............................] - ETA: 1:53 - loss: 75.9005 - mae: 6.7988
 3/95 [..............................] - ETA: 1:46 - loss: 87.0694 - mae: 7.2704
 4/95 [>.............................] - ETA: 1:39 - loss: 83.4590 - mae: 7.0993
 5/95 [>.............................] - ETA: 1:36 - loss: 80.7104 - mae: 7.0595
 6/95 [>.............................] - ETA: 1:35 - loss: 80.2990 - mae: 6.9392
 7/95 [=>............................] - ETA: 1:32 - loss: 81.2037 - mae: 6.9861
 8/95 [=>............................] - ETA: 1:29 - loss: 79.3207 - mae: 6.9410
 9/95 [=>............................] - ETA: 1:27 - loss: 80.8160 - mae: 7.0179
10/95 [==>...........................] - ETA: 1:25 - loss: 81.9162 - mae: 7.0850
11/95 [==>...........................] - ETA: 1:23 - loss: 81.9130 - mae: 7.1107
12/95 [==>...........................] - ETA: 1:22 - loss: 80.1762 - mae: 7.0151
13/95 [===>..........................] - ETA: 1:21 - loss: 82.2219 - mae: 7.0633
14/95 [===>..........................] - ETA: 1:19 - loss: 82.2622 - mae: 7.0760
15/95 [===>..........................] - ETA: 1:18 - loss: 81.7038 - mae: 7.0883
16/95 [====>.........................] - ETA: 1:17 - loss: 81.7421 - mae: 7.0752
17/95 [====>.........................] - ETA: 1:15 - loss: 81.4164 - mae: 7.0686
18/95 [====>.........................] - ETA: 1:14 - loss: 81.0592 - mae: 7.0453
19/95 [=====>........................] - ETA: 1:13 - loss: 81.3139 - mae: 7.0577
20/95 [=====>........................] - ETA: 1:12 - loss: 80.7799 - mae: 7.0506
21/95 [=====>........................] - ETA: 1:11 - loss: 81.3190 - mae: 7.0679
22/95 [=====>........................] - ETA: 1:10 - loss: 80.2201 - mae: 7.0064
23/95 [======>.......................] - ETA: 1:09 - loss: 79.8330 - mae: 6.9902
24/95 [======>.......................] - ETA: 1:08 - loss: 79.0499 - mae: 6.9459
25/95 [======>.......................] - ETA: 1:07 - loss: 78.5032 - mae: 6.9241
26/95 [=======>......................] - ETA: 1:06 - loss: 78.1267 - mae: 6.9082
27/95 [=======>......................] - ETA: 1:05 - loss: 77.5523 - mae: 6.8687
28/95 [=======>......................] - ETA: 1:04 - loss: 77.2830 - mae: 6.8387
29/95 [========>.....................] - ETA: 1:03 - loss: 76.3630 - mae: 6.8107
30/95 [========>.....................] - ETA: 1:02 - loss: 76.7296 - mae: 6.8205
31/95 [========>.....................] - ETA: 1:01 - loss: 76.8082 - mae: 6.8138
32/95 [=========>....................] - ETA: 1:00 - loss: 76.1223 - mae: 6.7797
33/95 [=========>....................] - ETA: 59s - loss: 76.6952 - mae: 6.8099 
34/95 [=========>....................] - ETA: 58s - loss: 76.2806 - mae: 6.7942
35/95 [==========>...................] - ETA: 57s - loss: 76.8566 - mae: 6.8218
36/95 [==========>...................] - ETA: 56s - loss: 76.3793 - mae: 6.8027
37/95 [==========>...................] - ETA: 55s - loss: 76.3470 - mae: 6.8035
38/95 [===========>..................] - ETA: 54s - loss: 77.5498 - mae: 6.8545
39/95 [===========>..................] - ETA: 53s - loss: 77.5502 - mae: 6.8558
40/95 [===========>..................] - ETA: 52s - loss: 77.3480 - mae: 6.8467
41/95 [===========>..................] - ETA: 51s - loss: 78.1436 - mae: 6.8869
42/95 [============>.................] - ETA: 50s - loss: 78.1708 - mae: 6.8772
43/95 [============>.................] - ETA: 49s - loss: 78.0703 - mae: 6.8679
44/95 [============>.................] - ETA: 48s - loss: 78.3921 - mae: 6.8846
45/95 [=============>................] - ETA: 47s - loss: 79.4070 - mae: 6.9106
46/95 [=============>................] - ETA: 46s - loss: 80.2238 - mae: 6.9319
47/95 [=============>................] - ETA: 45s - loss: 80.0669 - mae: 6.9278
48/95 [==============>...............] - ETA: 44s - loss: 79.4793 - mae: 6.9006
49/95 [==============>...............] - ETA: 43s - loss: 79.6633 - mae: 6.9017
50/95 [==============>...............] - ETA: 42s - loss: 79.6765 - mae: 6.9121
51/95 [===============>..............] - ETA: 41s - loss: 79.5272 - mae: 6.9146
52/95 [===============>..............] - ETA: 41s - loss: 79.7801 - mae: 6.9241
53/95 [===============>..............] - ETA: 40s - loss: 80.3194 - mae: 6.9425
54/95 [================>.............] - ETA: 39s - loss: 80.5467 - mae: 6.9392
55/95 [================>.............] - ETA: 38s - loss: 80.7623 - mae: 6.9586
56/95 [================>.............] - ETA: 37s - loss: 81.0782 - mae: 6.9645
57/95 [=================>............] - ETA: 36s - loss: 80.7643 - mae: 6.9475
58/95 [=================>............] - ETA: 35s - loss: 80.8909 - mae: 6.9572
59/95 [=================>............] - ETA: 34s - loss: 81.2174 - mae: 6.9703
60/95 [=================>............] - ETA: 33s - loss: 81.5991 - mae: 6.9796
61/95 [==================>...........] - ETA: 32s - loss: 81.5652 - mae: 6.9828
62/95 [==================>...........] - ETA: 31s - loss: 81.5464 - mae: 6.9864
63/95 [==================>...........] - ETA: 30s - loss: 81.6740 - mae: 6.9922
64/95 [===================>..........] - ETA: 29s - loss: 81.3019 - mae: 6.9679
65/95 [===================>..........] - ETA: 28s - loss: 81.1903 - mae: 6.9557
66/95 [===================>..........] - ETA: 27s - loss: 80.9298 - mae: 6.9496
67/95 [====================>.........] - ETA: 26s - loss: 80.5819 - mae: 6.9314
68/95 [====================>.........] - ETA: 25s - loss: 80.6433 - mae: 6.9319
69/95 [====================>.........] - ETA: 24s - loss: 80.8542 - mae: 6.9442
70/95 [=====================>........] - ETA: 23s - loss: 80.6567 - mae: 6.9354
71/95 [=====================>........] - ETA: 22s - loss: 81.0936 - mae: 6.9555
72/95 [=====================>........] - ETA: 21s - loss: 80.7925 - mae: 6.9475
73/95 [======================>.......] - ETA: 21s - loss: 80.5867 - mae: 6.9345
74/95 [======================>.......] - ETA: 20s - loss: 81.0334 - mae: 6.9381
75/95 [======================>.......] - ETA: 19s - loss: 81.2057 - mae: 6.9363
76/95 [=======================>......] - ETA: 18s - loss: 81.1257 - mae: 6.9315
77/95 [=======================>......] - ETA: 17s - loss: 81.1078 - mae: 6.9346
78/95 [=======================>......] - ETA: 16s - loss: 80.9664 - mae: 6.9265
79/95 [=======================>......] - ETA: 15s - loss: 80.8707 - mae: 6.9152
80/95 [========================>.....] - ETA: 14s - loss: 80.8015 - mae: 6.9103
81/95 [========================>.....] - ETA: 13s - loss: 80.9568 - mae: 6.9208
82/95 [========================>.....] - ETA: 12s - loss: 81.1678 - mae: 6.9257
83/95 [=========================>....] - ETA: 11s - loss: 81.1582 - mae: 6.9271
84/95 [=========================>....] - ETA: 10s - loss: 80.9165 - mae: 6.9149
85/95 [=========================>....] - ETA: 9s - loss: 80.6912 - mae: 6.9012 
86/95 [==========================>...] - ETA: 8s - loss: 81.1260 - mae: 6.9156
87/95 [==========================>...] - ETA: 7s - loss: 81.0015 - mae: 6.9100
88/95 [==========================>...] - ETA: 6s - loss: 80.4922 - mae: 6.8843
89/95 [===========================>..] - ETA: 5s - loss: 80.4967 - mae: 6.8893
90/95 [===========================>..] - ETA: 4s - loss: 80.8477 - mae: 6.9032
91/95 [===========================>..] - ETA: 3s - loss: 80.8086 - mae: 6.9017
92/95 [============================>.] - ETA: 2s - loss: 80.6869 - mae: 6.8926
93/95 [============================>.] - ETA: 1s - loss: 80.7426 - mae: 6.8986
94/95 [============================>.] - ETA: 0s - loss: 80.7013 - mae: 6.9022
95/95 [==============================] - 99s 1s/step - loss: 80.6268 - mae: 6.9031 - val_loss: 426.1253 - val_mae: 15.6464
Epoch 7/20

 1/95 [..............................] - ETA: 1:50 - loss: 67.6129 - mae: 5.5922
 2/95 [..............................] - ETA: 1:36 - loss: 81.6386 - mae: 6.6931
 3/95 [..............................] - ETA: 1:34 - loss: 77.4911 - mae: 6.6932
 4/95 [>.............................] - ETA: 1:31 - loss: 88.1229 - mae: 7.1070
 5/95 [>.............................] - ETA: 1:29 - loss: 86.1522 - mae: 7.0481
 6/95 [>.............................] - ETA: 1:26 - loss: 85.9541 - mae: 7.0512
 7/95 [=>............................] - ETA: 1:24 - loss: 92.5919 - mae: 7.1239
 8/95 [=>............................] - ETA: 1:23 - loss: 92.7502 - mae: 7.1527
 9/95 [=>............................] - ETA: 1:22 - loss: 89.4940 - mae: 7.0663
10/95 [==>...........................] - ETA: 1:21 - loss: 89.5738 - mae: 7.0867
11/95 [==>...........................] - ETA: 1:20 - loss: 88.8899 - mae: 7.0707
12/95 [==>...........................] - ETA: 1:19 - loss: 86.8515 - mae: 6.9748
13/95 [===>..........................] - ETA: 1:18 - loss: 85.9675 - mae: 6.9409
14/95 [===>..........................] - ETA: 1:17 - loss: 84.3451 - mae: 6.8639
15/95 [===>..........................] - ETA: 1:16 - loss: 83.6880 - mae: 6.8432
16/95 [====>.........................] - ETA: 1:15 - loss: 81.2899 - mae: 6.7599
17/95 [====>.........................] - ETA: 1:14 - loss: 81.8278 - mae: 6.7776
18/95 [====>.........................] - ETA: 1:13 - loss: 82.1270 - mae: 6.7974
19/95 [=====>........................] - ETA: 1:12 - loss: 81.9245 - mae: 6.8211
20/95 [=====>........................] - ETA: 1:11 - loss: 81.7601 - mae: 6.8124
21/95 [=====>........................] - ETA: 1:10 - loss: 83.0786 - mae: 6.8495
22/95 [=====>........................] - ETA: 1:09 - loss: 82.9493 - mae: 6.8423
23/95 [======>.......................] - ETA: 1:08 - loss: 81.8143 - mae: 6.8076
24/95 [======>.......................] - ETA: 1:07 - loss: 81.2392 - mae: 6.7824
25/95 [======>.......................] - ETA: 1:06 - loss: 81.0689 - mae: 6.7915
26/95 [=======>......................] - ETA: 1:05 - loss: 82.3942 - mae: 6.8410
27/95 [=======>......................] - ETA: 1:04 - loss: 82.1992 - mae: 6.8399
28/95 [=======>......................] - ETA: 1:03 - loss: 81.3184 - mae: 6.7975
29/95 [========>.....................] - ETA: 1:02 - loss: 80.4490 - mae: 6.7689
30/95 [========>.....................] - ETA: 1:01 - loss: 80.8693 - mae: 6.8004
31/95 [========>.....................] - ETA: 1:00 - loss: 81.8550 - mae: 6.8578
32/95 [=========>....................] - ETA: 59s - loss: 82.1524 - mae: 6.8852 
33/95 [=========>....................] - ETA: 58s - loss: 82.1084 - mae: 6.9010
34/95 [=========>....................] - ETA: 57s - loss: 82.3610 - mae: 6.8931
35/95 [==========>...................] - ETA: 57s - loss: 82.1394 - mae: 6.9026
36/95 [==========>...................] - ETA: 56s - loss: 82.2662 - mae: 6.9050
37/95 [==========>...................] - ETA: 55s - loss: 82.4484 - mae: 6.9144
38/95 [===========>..................] - ETA: 54s - loss: 82.4242 - mae: 6.9054
39/95 [===========>..................] - ETA: 53s - loss: 81.9328 - mae: 6.8738
40/95 [===========>..................] - ETA: 52s - loss: 81.3410 - mae: 6.8538
41/95 [===========>..................] - ETA: 51s - loss: 81.3571 - mae: 6.8562
42/95 [============>.................] - ETA: 50s - loss: 81.1079 - mae: 6.8464
43/95 [============>.................] - ETA: 49s - loss: 80.6422 - mae: 6.8278
44/95 [============>.................] - ETA: 48s - loss: 79.8352 - mae: 6.7981
45/95 [=============>................] - ETA: 47s - loss: 79.6275 - mae: 6.7968
46/95 [=============>................] - ETA: 46s - loss: 79.1485 - mae: 6.7813
47/95 [=============>................] - ETA: 45s - loss: 79.0138 - mae: 6.7861
48/95 [==============>...............] - ETA: 44s - loss: 79.3407 - mae: 6.7929
49/95 [==============>...............] - ETA: 43s - loss: 79.0709 - mae: 6.7895
50/95 [==============>...............] - ETA: 42s - loss: 78.9220 - mae: 6.7937
51/95 [===============>..............] - ETA: 41s - loss: 78.6229 - mae: 6.7916
52/95 [===============>..............] - ETA: 40s - loss: 78.7634 - mae: 6.7897
53/95 [===============>..............] - ETA: 39s - loss: 78.8692 - mae: 6.7925
54/95 [================>.............] - ETA: 38s - loss: 78.6299 - mae: 6.7807
55/95 [================>.............] - ETA: 38s - loss: 78.2324 - mae: 6.7642
56/95 [================>.............] - ETA: 37s - loss: 78.1575 - mae: 6.7643
57/95 [=================>............] - ETA: 36s - loss: 78.2556 - mae: 6.7743
58/95 [=================>............] - ETA: 35s - loss: 78.2115 - mae: 6.7694
59/95 [=================>............] - ETA: 34s - loss: 78.0852 - mae: 6.7680
60/95 [=================>............] - ETA: 33s - loss: 78.0831 - mae: 6.7672
61/95 [==================>...........] - ETA: 32s - loss: 78.0028 - mae: 6.7660
62/95 [==================>...........] - ETA: 31s - loss: 77.8824 - mae: 6.7598
63/95 [==================>...........] - ETA: 30s - loss: 77.5888 - mae: 6.7507
64/95 [===================>..........] - ETA: 29s - loss: 77.4069 - mae: 6.7492
65/95 [===================>..........] - ETA: 28s - loss: 77.5410 - mae: 6.7527
66/95 [===================>..........] - ETA: 27s - loss: 77.5529 - mae: 6.7540
67/95 [====================>.........] - ETA: 26s - loss: 77.5679 - mae: 6.7520
68/95 [====================>.........] - ETA: 25s - loss: 77.6420 - mae: 6.7562
69/95 [====================>.........] - ETA: 24s - loss: 77.4879 - mae: 6.7535
70/95 [=====================>........] - ETA: 23s - loss: 78.2681 - mae: 6.7797
71/95 [=====================>........] - ETA: 22s - loss: 77.8308 - mae: 6.7635
72/95 [=====================>........] - ETA: 21s - loss: 77.7860 - mae: 6.7638
73/95 [======================>.......] - ETA: 20s - loss: 78.1771 - mae: 6.7837
74/95 [======================>.......] - ETA: 19s - loss: 77.8783 - mae: 6.7640
75/95 [======================>.......] - ETA: 18s - loss: 77.7385 - mae: 6.7593
76/95 [=======================>......] - ETA: 18s - loss: 77.9592 - mae: 6.7699
77/95 [=======================>......] - ETA: 17s - loss: 77.9772 - mae: 6.7765
78/95 [=======================>......] - ETA: 16s - loss: 78.0264 - mae: 6.7770
79/95 [=======================>......] - ETA: 15s - loss: 78.0579 - mae: 6.7733
80/95 [========================>.....] - ETA: 14s - loss: 78.0672 - mae: 6.7735
81/95 [========================>.....] - ETA: 13s - loss: 78.2389 - mae: 6.7855
82/95 [========================>.....] - ETA: 12s - loss: 78.3754 - mae: 6.7921
83/95 [=========================>....] - ETA: 11s - loss: 78.4432 - mae: 6.7955
84/95 [=========================>....] - ETA: 10s - loss: 78.2943 - mae: 6.7906
85/95 [=========================>....] - ETA: 9s - loss: 77.8463 - mae: 6.7710 
86/95 [==========================>...] - ETA: 8s - loss: 77.5424 - mae: 6.7536
87/95 [==========================>...] - ETA: 7s - loss: 77.5506 - mae: 6.7575
88/95 [==========================>...] - ETA: 6s - loss: 77.2527 - mae: 6.7438
89/95 [===========================>..] - ETA: 5s - loss: 77.4029 - mae: 6.7498
90/95 [===========================>..] - ETA: 4s - loss: 77.4931 - mae: 6.7540
91/95 [===========================>..] - ETA: 3s - loss: 77.3055 - mae: 6.7447
92/95 [============================>.] - ETA: 2s - loss: 77.0380 - mae: 6.7332
93/95 [============================>.] - ETA: 1s - loss: 77.0736 - mae: 6.7321
94/95 [============================>.] - ETA: 0s - loss: 77.1060 - mae: 6.7317
95/95 [==============================] - 99s 1s/step - loss: 77.1368 - mae: 6.7317 - val_loss: 253.2360 - val_mae: 11.7148
Epoch 8/20

 1/95 [..............................] - ETA: 1:47 - loss: 61.5631 - mae: 5.6712
 2/95 [..............................] - ETA: 1:36 - loss: 67.7796 - mae: 6.2879
 3/95 [..............................] - ETA: 1:31 - loss: 75.8900 - mae: 6.5283
 4/95 [>.............................] - ETA: 1:31 - loss: 72.5412 - mae: 6.4299
 5/95 [>.............................] - ETA: 1:29 - loss: 72.0988 - mae: 6.4797
 6/95 [>.............................] - ETA: 1:27 - loss: 69.9426 - mae: 6.4464
 7/95 [=>............................] - ETA: 1:25 - loss: 74.1085 - mae: 6.6027
 8/95 [=>............................] - ETA: 1:24 - loss: 75.7725 - mae: 6.5900
 9/95 [=>............................] - ETA: 1:22 - loss: 74.9972 - mae: 6.5819
10/95 [==>...........................] - ETA: 1:21 - loss: 75.3906 - mae: 6.6234
11/95 [==>...........................] - ETA: 1:19 - loss: 74.2229 - mae: 6.5639
12/95 [==>...........................] - ETA: 1:20 - loss: 73.2236 - mae: 6.5454
13/95 [===>..........................] - ETA: 1:19 - loss: 72.7177 - mae: 6.5175
14/95 [===>..........................] - ETA: 1:18 - loss: 71.1672 - mae: 6.4568
15/95 [===>..........................] - ETA: 1:17 - loss: 71.8040 - mae: 6.4635
16/95 [====>.........................] - ETA: 1:16 - loss: 72.2537 - mae: 6.5067
17/95 [====>.........................] - ETA: 1:15 - loss: 72.1491 - mae: 6.4852
18/95 [====>.........................] - ETA: 1:13 - loss: 72.2195 - mae: 6.4765
19/95 [=====>........................] - ETA: 1:13 - loss: 71.6233 - mae: 6.4330
20/95 [=====>........................] - ETA: 1:13 - loss: 71.8514 - mae: 6.4485
21/95 [=====>........................] - ETA: 1:11 - loss: 72.1486 - mae: 6.4668
22/95 [=====>........................] - ETA: 1:11 - loss: 72.4143 - mae: 6.4713
23/95 [======>.......................] - ETA: 1:10 - loss: 71.5559 - mae: 6.4191
24/95 [======>.......................] - ETA: 1:09 - loss: 71.8806 - mae: 6.4613
25/95 [======>.......................] - ETA: 1:08 - loss: 71.4663 - mae: 6.4637
26/95 [=======>......................] - ETA: 1:07 - loss: 70.5232 - mae: 6.4316
27/95 [=======>......................] - ETA: 1:05 - loss: 70.8443 - mae: 6.4611
28/95 [=======>......................] - ETA: 1:04 - loss: 70.4031 - mae: 6.4399
29/95 [========>.....................] - ETA: 1:04 - loss: 70.9238 - mae: 6.4742
30/95 [========>.....................] - ETA: 1:03 - loss: 73.3457 - mae: 6.5127
31/95 [========>.....................] - ETA: 1:02 - loss: 73.7798 - mae: 6.5354
32/95 [=========>....................] - ETA: 1:01 - loss: 74.5625 - mae: 6.5681
33/95 [=========>....................] - ETA: 1:00 - loss: 73.3532 - mae: 6.5056
34/95 [=========>....................] - ETA: 59s - loss: 72.6387 - mae: 6.4563 
35/95 [==========>...................] - ETA: 58s - loss: 71.7177 - mae: 6.4170
36/95 [==========>...................] - ETA: 57s - loss: 71.5913 - mae: 6.4203
37/95 [==========>...................] - ETA: 56s - loss: 71.4568 - mae: 6.4168
38/95 [===========>..................] - ETA: 56s - loss: 72.1727 - mae: 6.4491
39/95 [===========>..................] - ETA: 55s - loss: 72.0067 - mae: 6.4503
40/95 [===========>..................] - ETA: 53s - loss: 72.3544 - mae: 6.4401
41/95 [===========>..................] - ETA: 53s - loss: 71.7338 - mae: 6.4152
42/95 [============>.................] - ETA: 52s - loss: 71.4853 - mae: 6.4026
43/95 [============>.................] - ETA: 51s - loss: 71.4241 - mae: 6.3983
44/95 [============>.................] - ETA: 50s - loss: 71.5693 - mae: 6.4118
45/95 [=============>................] - ETA: 49s - loss: 71.6973 - mae: 6.4254
46/95 [=============>................] - ETA: 48s - loss: 71.9167 - mae: 6.4412
47/95 [=============>................] - ETA: 47s - loss: 71.4353 - mae: 6.4204
48/95 [==============>...............] - ETA: 46s - loss: 71.5211 - mae: 6.4297
49/95 [==============>...............] - ETA: 45s - loss: 71.4186 - mae: 6.4335
50/95 [==============>...............] - ETA: 44s - loss: 71.0387 - mae: 6.4155
51/95 [===============>..............] - ETA: 43s - loss: 70.8002 - mae: 6.4093
52/95 [===============>..............] - ETA: 42s - loss: 70.2476 - mae: 6.3815
53/95 [===============>..............] - ETA: 41s - loss: 70.0331 - mae: 6.3717
54/95 [================>.............] - ETA: 40s - loss: 70.1144 - mae: 6.3800
55/95 [================>.............] - ETA: 39s - loss: 70.1794 - mae: 6.3694
56/95 [================>.............] - ETA: 38s - loss: 70.1210 - mae: 6.3715
57/95 [=================>............] - ETA: 37s - loss: 69.8552 - mae: 6.3608
58/95 [=================>............] - ETA: 36s - loss: 70.0917 - mae: 6.3772
59/95 [=================>............] - ETA: 35s - loss: 70.2384 - mae: 6.3803
60/95 [=================>............] - ETA: 34s - loss: 71.3196 - mae: 6.4109
61/95 [==================>...........] - ETA: 33s - loss: 71.5749 - mae: 6.4202
62/95 [==================>...........] - ETA: 32s - loss: 71.6271 - mae: 6.4234
63/95 [==================>...........] - ETA: 31s - loss: 71.7378 - mae: 6.4314
64/95 [===================>..........] - ETA: 30s - loss: 71.7308 - mae: 6.4406
65/95 [===================>..........] - ETA: 29s - loss: 71.7289 - mae: 6.4446
66/95 [===================>..........] - ETA: 28s - loss: 71.5410 - mae: 6.4432
67/95 [====================>.........] - ETA: 27s - loss: 71.5399 - mae: 6.4462
68/95 [====================>.........] - ETA: 26s - loss: 72.1336 - mae: 6.4726
69/95 [====================>.........] - ETA: 25s - loss: 72.1827 - mae: 6.4758
70/95 [=====================>........] - ETA: 24s - loss: 72.1964 - mae: 6.4812
71/95 [=====================>........] - ETA: 23s - loss: 71.7961 - mae: 6.4688
72/95 [=====================>........] - ETA: 22s - loss: 71.6785 - mae: 6.4639
73/95 [======================>.......] - ETA: 21s - loss: 71.6152 - mae: 6.4640
74/95 [======================>.......] - ETA: 20s - loss: 71.9571 - mae: 6.4732
75/95 [======================>.......] - ETA: 19s - loss: 71.7657 - mae: 6.4706
76/95 [=======================>......] - ETA: 18s - loss: 72.1558 - mae: 6.4935
77/95 [=======================>......] - ETA: 17s - loss: 72.2066 - mae: 6.4970
78/95 [=======================>......] - ETA: 16s - loss: 72.3671 - mae: 6.5024
79/95 [=======================>......] - ETA: 15s - loss: 72.3110 - mae: 6.5010
80/95 [========================>.....] - ETA: 14s - loss: 72.3271 - mae: 6.5059
81/95 [========================>.....] - ETA: 13s - loss: 72.3667 - mae: 6.5063
82/95 [========================>.....] - ETA: 12s - loss: 72.1309 - mae: 6.4941
83/95 [=========================>....] - ETA: 11s - loss: 72.2456 - mae: 6.4958
84/95 [=========================>....] - ETA: 10s - loss: 72.2024 - mae: 6.4930
85/95 [=========================>....] - ETA: 9s - loss: 72.2188 - mae: 6.4931 
86/95 [==========================>...] - ETA: 8s - loss: 72.3903 - mae: 6.4988
87/95 [==========================>...] - ETA: 7s - loss: 72.4536 - mae: 6.4960
88/95 [==========================>...] - ETA: 6s - loss: 72.4493 - mae: 6.4959
89/95 [===========================>..] - ETA: 5s - loss: 72.3845 - mae: 6.4964
90/95 [===========================>..] - ETA: 4s - loss: 72.2594 - mae: 6.4866
91/95 [===========================>..] - ETA: 3s - loss: 72.5074 - mae: 6.4979
92/95 [============================>.] - ETA: 2s - loss: 72.6192 - mae: 6.5023
93/95 [============================>.] - ETA: 1s - loss: 72.5932 - mae: 6.5012
94/95 [============================>.] - ETA: 0s - loss: 72.4833 - mae: 6.4976
95/95 [==============================] - 99s 1s/step - loss: 72.4792 - mae: 6.4965 - val_loss: 124.1216 - val_mae: 8.3135
Epoch 9/20

 1/95 [..............................] - ETA: 1:52 - loss: 64.3101 - mae: 6.1068
 2/95 [..............................] - ETA: 1:54 - loss: 61.0471 - mae: 6.0047
 3/95 [..............................] - ETA: 1:43 - loss: 61.5250 - mae: 6.0090
 4/95 [>.............................] - ETA: 1:43 - loss: 63.2361 - mae: 6.1254
 5/95 [>.............................] - ETA: 1:39 - loss: 63.6770 - mae: 6.1208
 6/95 [>.............................] - ETA: 1:35 - loss: 62.8997 - mae: 6.0955
 7/95 [=>............................] - ETA: 1:32 - loss: 57.8319 - mae: 5.8128
 8/95 [=>............................] - ETA: 1:30 - loss: 56.5944 - mae: 5.7790
 9/95 [=>............................] - ETA: 1:29 - loss: 60.3382 - mae: 5.8826
10/95 [==>...........................] - ETA: 1:28 - loss: 60.7430 - mae: 5.9729
11/95 [==>...........................] - ETA: 1:26 - loss: 65.4411 - mae: 6.0396
12/95 [==>...........................] - ETA: 1:24 - loss: 69.7210 - mae: 6.2224
13/95 [===>..........................] - ETA: 1:23 - loss: 70.3873 - mae: 6.2590
14/95 [===>..........................] - ETA: 1:22 - loss: 67.9809 - mae: 6.1460
15/95 [===>..........................] - ETA: 1:20 - loss: 68.6056 - mae: 6.1774
16/95 [====>.........................] - ETA: 1:19 - loss: 69.1846 - mae: 6.2180
17/95 [====>.........................] - ETA: 1:17 - loss: 67.1778 - mae: 6.1285
18/95 [====>.........................] - ETA: 1:16 - loss: 68.2130 - mae: 6.2047
19/95 [=====>........................] - ETA: 1:15 - loss: 67.5355 - mae: 6.1819
20/95 [=====>........................] - ETA: 1:14 - loss: 67.6789 - mae: 6.2139
21/95 [=====>........................] - ETA: 1:12 - loss: 67.6562 - mae: 6.2314
22/95 [=====>........................] - ETA: 1:11 - loss: 68.2867 - mae: 6.2661
23/95 [======>.......................] - ETA: 1:10 - loss: 68.8367 - mae: 6.3032
24/95 [======>.......................] - ETA: 1:09 - loss: 68.4219 - mae: 6.3019
25/95 [======>.......................] - ETA: 1:08 - loss: 68.8896 - mae: 6.3393
26/95 [=======>......................] - ETA: 1:07 - loss: 68.2464 - mae: 6.2963
27/95 [=======>......................] - ETA: 1:06 - loss: 67.9312 - mae: 6.2816
28/95 [=======>......................] - ETA: 1:05 - loss: 68.0492 - mae: 6.2948
29/95 [========>.....................] - ETA: 1:04 - loss: 67.1769 - mae: 6.2580
30/95 [========>.....................] - ETA: 1:03 - loss: 67.4827 - mae: 6.2801
31/95 [========>.....................] - ETA: 1:02 - loss: 67.1708 - mae: 6.2824
32/95 [=========>....................] - ETA: 1:01 - loss: 68.0381 - mae: 6.3231
33/95 [=========>....................] - ETA: 1:00 - loss: 67.7871 - mae: 6.3094
34/95 [=========>....................] - ETA: 59s - loss: 67.7851 - mae: 6.3089 
35/95 [==========>...................] - ETA: 58s - loss: 67.3408 - mae: 6.3009
36/95 [==========>...................] - ETA: 57s - loss: 67.5640 - mae: 6.3116
37/95 [==========>...................] - ETA: 56s - loss: 68.4261 - mae: 6.3488
38/95 [===========>..................] - ETA: 55s - loss: 68.1403 - mae: 6.3396
39/95 [===========>..................] - ETA: 54s - loss: 68.3060 - mae: 6.3521
40/95 [===========>..................] - ETA: 53s - loss: 68.0939 - mae: 6.3454
41/95 [===========>..................] - ETA: 52s - loss: 67.8946 - mae: 6.3342
42/95 [============>.................] - ETA: 51s - loss: 67.5655 - mae: 6.3277
43/95 [============>.................] - ETA: 50s - loss: 67.6928 - mae: 6.3277
44/95 [============>.................] - ETA: 49s - loss: 67.4458 - mae: 6.3166
45/95 [=============>................] - ETA: 48s - loss: 67.0882 - mae: 6.3031
46/95 [=============>................] - ETA: 47s - loss: 67.4564 - mae: 6.3237
47/95 [=============>................] - ETA: 46s - loss: 67.2511 - mae: 6.3147
48/95 [==============>...............] - ETA: 45s - loss: 67.5163 - mae: 6.3269
49/95 [==============>...............] - ETA: 44s - loss: 67.9192 - mae: 6.3483
50/95 [==============>...............] - ETA: 43s - loss: 67.6028 - mae: 6.3334
51/95 [===============>..............] - ETA: 42s - loss: 67.5943 - mae: 6.3331
52/95 [===============>..............] - ETA: 41s - loss: 68.2507 - mae: 6.3475
53/95 [===============>..............] - ETA: 40s - loss: 68.2841 - mae: 6.3510
54/95 [================>.............] - ETA: 39s - loss: 68.1437 - mae: 6.3416
55/95 [================>.............] - ETA: 38s - loss: 68.3254 - mae: 6.3495
56/95 [================>.............] - ETA: 37s - loss: 68.3480 - mae: 6.3476
57/95 [=================>............] - ETA: 36s - loss: 68.4174 - mae: 6.3598
58/95 [=================>............] - ETA: 35s - loss: 68.1677 - mae: 6.3445
59/95 [=================>............] - ETA: 34s - loss: 68.1855 - mae: 6.3526
60/95 [=================>............] - ETA: 33s - loss: 68.4688 - mae: 6.3694
61/95 [==================>...........] - ETA: 33s - loss: 68.3483 - mae: 6.3671
62/95 [==================>...........] - ETA: 32s - loss: 68.6054 - mae: 6.3840
63/95 [==================>...........] - ETA: 31s - loss: 68.4542 - mae: 6.3776
64/95 [===================>..........] - ETA: 30s - loss: 68.3211 - mae: 6.3667
65/95 [===================>..........] - ETA: 29s - loss: 68.1683 - mae: 6.3651
66/95 [===================>..........] - ETA: 28s - loss: 67.9353 - mae: 6.3526
67/95 [====================>.........] - ETA: 27s - loss: 67.6224 - mae: 6.3378
68/95 [====================>.........] - ETA: 26s - loss: 67.7522 - mae: 6.3436
69/95 [====================>.........] - ETA: 25s - loss: 67.9874 - mae: 6.3551
70/95 [=====================>........] - ETA: 24s - loss: 67.9346 - mae: 6.3543
71/95 [=====================>........] - ETA: 23s - loss: 68.0908 - mae: 6.3638
72/95 [=====================>........] - ETA: 22s - loss: 67.9579 - mae: 6.3571
73/95 [======================>.......] - ETA: 21s - loss: 67.8851 - mae: 6.3565
74/95 [======================>.......] - ETA: 20s - loss: 68.6523 - mae: 6.3826
75/95 [======================>.......] - ETA: 19s - loss: 68.6245 - mae: 6.3724
76/95 [=======================>......] - ETA: 18s - loss: 68.5790 - mae: 6.3709
77/95 [=======================>......] - ETA: 17s - loss: 68.4721 - mae: 6.3677
78/95 [=======================>......] - ETA: 16s - loss: 68.3127 - mae: 6.3579
79/95 [=======================>......] - ETA: 15s - loss: 68.2520 - mae: 6.3574
80/95 [========================>.....] - ETA: 14s - loss: 68.2987 - mae: 6.3550
81/95 [========================>.....] - ETA: 13s - loss: 68.2851 - mae: 6.3575
82/95 [========================>.....] - ETA: 12s - loss: 68.0483 - mae: 6.3497
83/95 [=========================>....] - ETA: 11s - loss: 67.8346 - mae: 6.3388
84/95 [=========================>....] - ETA: 10s - loss: 67.8845 - mae: 6.3365
85/95 [=========================>....] - ETA: 9s - loss: 67.7917 - mae: 6.3290 
86/95 [==========================>...] - ETA: 8s - loss: 67.6998 - mae: 6.3276
87/95 [==========================>...] - ETA: 7s - loss: 67.8933 - mae: 6.3318
88/95 [==========================>...] - ETA: 6s - loss: 67.7177 - mae: 6.3221
89/95 [===========================>..] - ETA: 5s - loss: 67.7803 - mae: 6.3287
90/95 [===========================>..] - ETA: 4s - loss: 67.5893 - mae: 6.3182
91/95 [===========================>..] - ETA: 3s - loss: 67.7742 - mae: 6.3193
92/95 [============================>.] - ETA: 2s - loss: 67.5466 - mae: 6.3097
93/95 [============================>.] - ETA: 1s - loss: 67.5506 - mae: 6.3074
94/95 [============================>.] - ETA: 0s - loss: 67.2991 - mae: 6.2942
95/95 [==============================] - 100s 1s/step - loss: 67.2195 - mae: 6.2925 - val_loss: 93.1411 - val_mae: 7.1172
Epoch 10/20

 1/95 [..............................] - ETA: 1:49 - loss: 41.0054 - mae: 4.6852
 2/95 [..............................] - ETA: 1:41 - loss: 46.7074 - mae: 5.1283
 3/95 [..............................] - ETA: 1:36 - loss: 48.0391 - mae: 5.2546
 4/95 [>.............................] - ETA: 1:33 - loss: 49.0956 - mae: 5.2269
 5/95 [>.............................] - ETA: 1:30 - loss: 48.7958 - mae: 5.2887
 6/95 [>.............................] - ETA: 1:29 - loss: 50.4884 - mae: 5.4491
 7/95 [=>............................] - ETA: 1:26 - loss: 53.2307 - mae: 5.5221
 8/95 [=>............................] - ETA: 1:25 - loss: 53.4105 - mae: 5.5266
 9/95 [=>............................] - ETA: 1:26 - loss: 53.3831 - mae: 5.5577
10/95 [==>...........................] - ETA: 1:24 - loss: 53.2616 - mae: 5.5646
11/95 [==>...........................] - ETA: 1:22 - loss: 55.1375 - mae: 5.7058
12/95 [==>...........................] - ETA: 1:22 - loss: 54.2970 - mae: 5.6593
13/95 [===>..........................] - ETA: 1:20 - loss: 54.8852 - mae: 5.6487
14/95 [===>..........................] - ETA: 1:19 - loss: 55.1029 - mae: 5.6694
15/95 [===>..........................] - ETA: 1:18 - loss: 55.3750 - mae: 5.6903
16/95 [====>.........................] - ETA: 1:17 - loss: 55.5385 - mae: 5.7008
17/95 [====>.........................] - ETA: 1:15 - loss: 55.4427 - mae: 5.6966
18/95 [====>.........................] - ETA: 1:14 - loss: 55.4492 - mae: 5.7045
19/95 [=====>........................] - ETA: 1:13 - loss: 55.3123 - mae: 5.7317
20/95 [=====>........................] - ETA: 1:12 - loss: 55.2172 - mae: 5.7381
21/95 [=====>........................] - ETA: 1:11 - loss: 55.4983 - mae: 5.7447
22/95 [=====>........................] - ETA: 1:10 - loss: 56.0139 - mae: 5.7779
23/95 [======>.......................] - ETA: 1:09 - loss: 56.1683 - mae: 5.7953
24/95 [======>.......................] - ETA: 1:08 - loss: 55.8882 - mae: 5.7778
25/95 [======>.......................] - ETA: 1:07 - loss: 55.7125 - mae: 5.7591
26/95 [=======>......................] - ETA: 1:07 - loss: 55.7454 - mae: 5.7460
27/95 [=======>......................] - ETA: 1:05 - loss: 56.0694 - mae: 5.7638
28/95 [=======>......................] - ETA: 1:04 - loss: 56.1800 - mae: 5.7666
29/95 [========>.....................] - ETA: 1:03 - loss: 56.1810 - mae: 5.7603
30/95 [========>.....................] - ETA: 1:02 - loss: 56.6552 - mae: 5.7899
31/95 [========>.....................] - ETA: 1:01 - loss: 57.5733 - mae: 5.8166
32/95 [=========>....................] - ETA: 1:00 - loss: 57.9280 - mae: 5.8196
33/95 [=========>....................] - ETA: 59s - loss: 58.1434 - mae: 5.8378 
34/95 [=========>....................] - ETA: 58s - loss: 58.1183 - mae: 5.8263
35/95 [==========>...................] - ETA: 57s - loss: 57.7223 - mae: 5.8057
36/95 [==========>...................] - ETA: 56s - loss: 57.4784 - mae: 5.7981
37/95 [==========>...................] - ETA: 55s - loss: 57.7707 - mae: 5.8088
38/95 [===========>..................] - ETA: 54s - loss: 58.0240 - mae: 5.8166
39/95 [===========>..................] - ETA: 53s - loss: 57.9742 - mae: 5.8095
40/95 [===========>..................] - ETA: 52s - loss: 57.7885 - mae: 5.8052
41/95 [===========>..................] - ETA: 51s - loss: 58.1870 - mae: 5.8338
42/95 [============>.................] - ETA: 50s - loss: 57.8520 - mae: 5.8305
43/95 [============>.................] - ETA: 49s - loss: 57.6611 - mae: 5.8291
44/95 [============>.................] - ETA: 48s - loss: 57.8166 - mae: 5.8371
45/95 [=============>................] - ETA: 47s - loss: 57.8875 - mae: 5.8418
46/95 [=============>................] - ETA: 46s - loss: 57.4694 - mae: 5.8228
47/95 [=============>................] - ETA: 45s - loss: 57.5036 - mae: 5.8219
48/95 [==============>...............] - ETA: 44s - loss: 57.7435 - mae: 5.8356
49/95 [==============>...............] - ETA: 43s - loss: 57.6235 - mae: 5.8378
50/95 [==============>...............] - ETA: 42s - loss: 57.5826 - mae: 5.8317
51/95 [===============>..............] - ETA: 41s - loss: 57.5793 - mae: 5.8350
52/95 [===============>..............] - ETA: 40s - loss: 57.4104 - mae: 5.8273
53/95 [===============>..............] - ETA: 39s - loss: 57.5936 - mae: 5.8435
54/95 [================>.............] - ETA: 38s - loss: 57.4042 - mae: 5.8320
55/95 [================>.............] - ETA: 38s - loss: 57.1918 - mae: 5.8296
56/95 [================>.............] - ETA: 37s - loss: 56.9274 - mae: 5.8188
57/95 [=================>............] - ETA: 36s - loss: 57.4081 - mae: 5.8420
58/95 [=================>............] - ETA: 35s - loss: 57.8153 - mae: 5.8478
59/95 [=================>............] - ETA: 34s - loss: 58.4527 - mae: 5.8669
60/95 [=================>............] - ETA: 33s - loss: 58.4047 - mae: 5.8581
61/95 [==================>...........] - ETA: 32s - loss: 58.5305 - mae: 5.8575
62/95 [==================>...........] - ETA: 31s - loss: 58.7069 - mae: 5.8649
63/95 [==================>...........] - ETA: 30s - loss: 58.5900 - mae: 5.8639
64/95 [===================>..........] - ETA: 29s - loss: 58.4804 - mae: 5.8570
65/95 [===================>..........] - ETA: 28s - loss: 58.7124 - mae: 5.8660
66/95 [===================>..........] - ETA: 27s - loss: 58.5961 - mae: 5.8673
67/95 [====================>.........] - ETA: 26s - loss: 58.6337 - mae: 5.8737
68/95 [====================>.........] - ETA: 25s - loss: 58.3989 - mae: 5.8654
69/95 [====================>.........] - ETA: 24s - loss: 58.6899 - mae: 5.8860
70/95 [=====================>........] - ETA: 23s - loss: 59.0469 - mae: 5.8873
71/95 [=====================>........] - ETA: 22s - loss: 59.2325 - mae: 5.8973
72/95 [=====================>........] - ETA: 21s - loss: 59.2700 - mae: 5.9057
73/95 [======================>.......] - ETA: 20s - loss: 59.3338 - mae: 5.9086
74/95 [======================>.......] - ETA: 19s - loss: 58.9643 - mae: 5.8886
75/95 [======================>.......] - ETA: 18s - loss: 59.2526 - mae: 5.9022
76/95 [=======================>......] - ETA: 17s - loss: 59.0377 - mae: 5.8923
77/95 [=======================>......] - ETA: 17s - loss: 58.9904 - mae: 5.8918
78/95 [=======================>......] - ETA: 16s - loss: 58.8455 - mae: 5.8869
79/95 [=======================>......] - ETA: 15s - loss: 58.8467 - mae: 5.8869
80/95 [========================>.....] - ETA: 14s - loss: 58.9253 - mae: 5.8917
81/95 [========================>.....] - ETA: 13s - loss: 58.9920 - mae: 5.9003
82/95 [========================>.....] - ETA: 12s - loss: 58.9291 - mae: 5.8962
83/95 [=========================>....] - ETA: 11s - loss: 58.8949 - mae: 5.8979
84/95 [=========================>....] - ETA: 10s - loss: 59.1879 - mae: 5.9094
85/95 [=========================>....] - ETA: 9s - loss: 59.1365 - mae: 5.9084 
86/95 [==========================>...] - ETA: 8s - loss: 59.4418 - mae: 5.9237
87/95 [==========================>...] - ETA: 7s - loss: 59.6517 - mae: 5.9338
88/95 [==========================>...] - ETA: 6s - loss: 59.6152 - mae: 5.9362
89/95 [===========================>..] - ETA: 5s - loss: 59.6530 - mae: 5.9374
90/95 [===========================>..] - ETA: 4s - loss: 59.7188 - mae: 5.9400
91/95 [===========================>..] - ETA: 3s - loss: 59.8281 - mae: 5.9390
92/95 [============================>.] - ETA: 2s - loss: 59.7677 - mae: 5.9374
93/95 [============================>.] - ETA: 1s - loss: 60.0381 - mae: 5.9472
94/95 [============================>.] - ETA: 0s - loss: 59.9411 - mae: 5.9447
95/95 [==============================] - 99s 1s/step - loss: 59.9866 - mae: 5.9481 - val_loss: 93.5471 - val_mae: 7.1741
Epoch 11/20

 1/95 [..............................] - ETA: 1:53 - loss: 66.2652 - mae: 6.6221
 2/95 [..............................] - ETA: 1:38 - loss: 76.0192 - mae: 7.0302
 3/95 [..............................] - ETA: 1:33 - loss: 69.0602 - mae: 6.6194
 4/95 [>.............................] - ETA: 1:29 - loss: 65.7403 - mae: 6.4873
 5/95 [>.............................] - ETA: 1:29 - loss: 67.0770 - mae: 6.5344
 6/95 [>.............................] - ETA: 1:27 - loss: 64.9999 - mae: 6.4364
 7/95 [=>............................] - ETA: 1:27 - loss: 62.2674 - mae: 6.2327
 8/95 [=>............................] - ETA: 1:25 - loss: 62.7981 - mae: 6.2018
 9/95 [=>............................] - ETA: 1:24 - loss: 62.8521 - mae: 6.1945
10/95 [==>...........................] - ETA: 1:22 - loss: 63.7011 - mae: 6.1850
11/95 [==>...........................] - ETA: 1:21 - loss: 62.7472 - mae: 6.0990
12/95 [==>...........................] - ETA: 1:20 - loss: 62.2510 - mae: 6.0737
13/95 [===>..........................] - ETA: 1:19 - loss: 61.1626 - mae: 6.0024
14/95 [===>..........................] - ETA: 1:18 - loss: 60.3053 - mae: 5.9808
15/95 [===>..........................] - ETA: 1:17 - loss: 59.0419 - mae: 5.9032
16/95 [====>.........................] - ETA: 1:16 - loss: 58.8126 - mae: 5.8777
17/95 [====>.........................] - ETA: 1:15 - loss: 57.8923 - mae: 5.8512
18/95 [====>.........................] - ETA: 1:14 - loss: 58.0330 - mae: 5.8635
19/95 [=====>........................] - ETA: 1:13 - loss: 57.7159 - mae: 5.8688
20/95 [=====>........................] - ETA: 1:12 - loss: 57.4457 - mae: 5.8651
21/95 [=====>........................] - ETA: 1:11 - loss: 56.8937 - mae: 5.8417
22/95 [=====>........................] - ETA: 1:09 - loss: 57.7713 - mae: 5.8258
23/95 [======>.......................] - ETA: 1:08 - loss: 57.5930 - mae: 5.8207
24/95 [======>.......................] - ETA: 1:07 - loss: 57.1610 - mae: 5.7973
25/95 [======>.......................] - ETA: 1:06 - loss: 56.7978 - mae: 5.7953
26/95 [=======>......................] - ETA: 1:05 - loss: 56.7402 - mae: 5.7829
27/95 [=======>......................] - ETA: 1:04 - loss: 56.9600 - mae: 5.8029
28/95 [=======>......................] - ETA: 1:03 - loss: 56.4131 - mae: 5.7705
29/95 [========>.....................] - ETA: 1:03 - loss: 57.1057 - mae: 5.7929
30/95 [========>.....................] - ETA: 1:01 - loss: 57.1320 - mae: 5.8046
31/95 [========>.....................] - ETA: 1:01 - loss: 57.1768 - mae: 5.8031
32/95 [=========>....................] - ETA: 1:00 - loss: 57.5901 - mae: 5.8155
33/95 [=========>....................] - ETA: 59s - loss: 57.5706 - mae: 5.8197 
34/95 [=========>....................] - ETA: 58s - loss: 57.6630 - mae: 5.8250
35/95 [==========>...................] - ETA: 57s - loss: 57.4197 - mae: 5.8170
36/95 [==========>...................] - ETA: 56s - loss: 58.4525 - mae: 5.8614
37/95 [==========>...................] - ETA: 55s - loss: 58.3884 - mae: 5.8587
38/95 [===========>..................] - ETA: 54s - loss: 58.1642 - mae: 5.8579
39/95 [===========>..................] - ETA: 53s - loss: 58.5941 - mae: 5.8803
40/95 [===========>..................] - ETA: 52s - loss: 58.5572 - mae: 5.8872
41/95 [===========>..................] - ETA: 51s - loss: 58.3317 - mae: 5.8759
42/95 [============>.................] - ETA: 50s - loss: 58.3404 - mae: 5.8818
43/95 [============>.................] - ETA: 49s - loss: 58.1513 - mae: 5.8650
44/95 [============>.................] - ETA: 48s - loss: 58.1521 - mae: 5.8555
45/95 [=============>................] - ETA: 47s - loss: 58.3997 - mae: 5.8697
46/95 [=============>................] - ETA: 46s - loss: 58.3046 - mae: 5.8654
47/95 [=============>................] - ETA: 45s - loss: 58.3459 - mae: 5.8637
48/95 [==============>...............] - ETA: 44s - loss: 58.3170 - mae: 5.8584
49/95 [==============>...............] - ETA: 43s - loss: 58.1854 - mae: 5.8585
50/95 [==============>...............] - ETA: 42s - loss: 58.5174 - mae: 5.8798
51/95 [===============>..............] - ETA: 42s - loss: 58.6440 - mae: 5.8886
52/95 [===============>..............] - ETA: 41s - loss: 58.7413 - mae: 5.8903
53/95 [===============>..............] - ETA: 40s - loss: 58.3761 - mae: 5.8772
54/95 [================>.............] - ETA: 39s - loss: 58.3915 - mae: 5.8769
55/95 [================>.............] - ETA: 38s - loss: 58.2878 - mae: 5.8757
56/95 [================>.............] - ETA: 37s - loss: 58.9400 - mae: 5.9012
57/95 [=================>............] - ETA: 36s - loss: 58.9398 - mae: 5.9035
58/95 [=================>............] - ETA: 35s - loss: 58.8692 - mae: 5.9033
59/95 [=================>............] - ETA: 34s - loss: 59.0473 - mae: 5.9148
60/95 [=================>............] - ETA: 33s - loss: 58.8739 - mae: 5.9103
61/95 [==================>...........] - ETA: 32s - loss: 58.8637 - mae: 5.9146
62/95 [==================>...........] - ETA: 31s - loss: 59.4437 - mae: 5.9279
63/95 [==================>...........] - ETA: 30s - loss: 59.4298 - mae: 5.9193
64/95 [===================>..........] - ETA: 29s - loss: 59.1605 - mae: 5.9009
65/95 [===================>..........] - ETA: 28s - loss: 59.3009 - mae: 5.9044
66/95 [===================>..........] - ETA: 27s - loss: 59.1913 - mae: 5.8992
67/95 [====================>.........] - ETA: 26s - loss: 58.8761 - mae: 5.8835
68/95 [====================>.........] - ETA: 25s - loss: 58.8075 - mae: 5.8825
69/95 [====================>.........] - ETA: 24s - loss: 58.9468 - mae: 5.8893
70/95 [=====================>........] - ETA: 23s - loss: 58.8492 - mae: 5.8815
71/95 [=====================>........] - ETA: 22s - loss: 58.6887 - mae: 5.8753
72/95 [=====================>........] - ETA: 21s - loss: 58.7159 - mae: 5.8733
73/95 [======================>.......] - ETA: 21s - loss: 58.9089 - mae: 5.8860
74/95 [======================>.......] - ETA: 20s - loss: 59.1310 - mae: 5.8963
75/95 [======================>.......] - ETA: 19s - loss: 59.2292 - mae: 5.9039
76/95 [=======================>......] - ETA: 18s - loss: 59.1439 - mae: 5.8968
77/95 [=======================>......] - ETA: 17s - loss: 59.0681 - mae: 5.8940
78/95 [=======================>......] - ETA: 16s - loss: 59.2032 - mae: 5.8997
79/95 [=======================>......] - ETA: 15s - loss: 59.3698 - mae: 5.9041
80/95 [========================>.....] - ETA: 14s - loss: 59.4290 - mae: 5.9091
81/95 [========================>.....] - ETA: 13s - loss: 59.2366 - mae: 5.8974
82/95 [========================>.....] - ETA: 12s - loss: 59.1148 - mae: 5.8916
83/95 [=========================>....] - ETA: 11s - loss: 58.8270 - mae: 5.8802
84/95 [=========================>....] - ETA: 10s - loss: 58.7512 - mae: 5.8748
85/95 [=========================>....] - ETA: 9s - loss: 58.7165 - mae: 5.8727 
86/95 [==========================>...] - ETA: 8s - loss: 58.6610 - mae: 5.8747
87/95 [==========================>...] - ETA: 7s - loss: 58.8286 - mae: 5.8800
88/95 [==========================>...] - ETA: 6s - loss: 58.5160 - mae: 5.8640
89/95 [===========================>..] - ETA: 5s - loss: 58.4758 - mae: 5.8648
90/95 [===========================>..] - ETA: 4s - loss: 58.4938 - mae: 5.8623
91/95 [===========================>..] - ETA: 3s - loss: 58.8756 - mae: 5.8719
92/95 [============================>.] - ETA: 2s - loss: 58.7804 - mae: 5.8678
93/95 [============================>.] - ETA: 1s - loss: 58.9324 - mae: 5.8777
94/95 [============================>.] - ETA: 0s - loss: 58.9179 - mae: 5.8782
95/95 [==============================] - 99s 1s/step - loss: 59.0541 - mae: 5.8851 - val_loss: 111.0918 - val_mae: 7.7886
Epoch 12/20

 1/95 [..............................] - ETA: 1:46 - loss: 57.9629 - mae: 5.4930
 2/95 [..............................] - ETA: 1:37 - loss: 61.1605 - mae: 5.9407
 3/95 [..............................] - ETA: 1:31 - loss: 63.4196 - mae: 6.2509
 4/95 [>.............................] - ETA: 1:29 - loss: 66.3181 - mae: 6.3723
 5/95 [>.............................] - ETA: 1:26 - loss: 61.3943 - mae: 6.0939
 6/95 [>.............................] - ETA: 1:25 - loss: 62.9437 - mae: 6.1810
 7/95 [=>............................] - ETA: 1:23 - loss: 62.9483 - mae: 6.1979
 8/95 [=>............................] - ETA: 1:22 - loss: 62.2246 - mae: 6.2138
 9/95 [=>............................] - ETA: 1:20 - loss: 61.7913 - mae: 6.1885
10/95 [==>...........................] - ETA: 1:20 - loss: 59.6736 - mae: 6.0585
11/95 [==>...........................] - ETA: 1:19 - loss: 57.2004 - mae: 5.8596
12/95 [==>...........................] - ETA: 1:18 - loss: 57.6748 - mae: 5.8707
13/95 [===>..........................] - ETA: 1:17 - loss: 57.8309 - mae: 5.8662
14/95 [===>..........................] - ETA: 1:16 - loss: 57.2235 - mae: 5.8442
15/95 [===>..........................] - ETA: 1:15 - loss: 59.0619 - mae: 5.9055
16/95 [====>.........................] - ETA: 1:14 - loss: 58.4525 - mae: 5.8719
17/95 [====>.........................] - ETA: 1:13 - loss: 57.9044 - mae: 5.8412
18/95 [====>.........................] - ETA: 1:12 - loss: 59.2460 - mae: 5.9014
19/95 [=====>........................] - ETA: 1:12 - loss: 59.3071 - mae: 5.9087
20/95 [=====>........................] - ETA: 1:11 - loss: 58.3548 - mae: 5.8467
21/95 [=====>........................] - ETA: 1:10 - loss: 57.8823 - mae: 5.7999
22/95 [=====>........................] - ETA: 1:09 - loss: 58.1813 - mae: 5.8193
23/95 [======>.......................] - ETA: 1:08 - loss: 57.4472 - mae: 5.7804
24/95 [======>.......................] - ETA: 1:07 - loss: 57.0170 - mae: 5.7667
25/95 [======>.......................] - ETA: 1:06 - loss: 56.7533 - mae: 5.7608
26/95 [=======>......................] - ETA: 1:05 - loss: 56.6813 - mae: 5.7666
27/95 [=======>......................] - ETA: 1:04 - loss: 56.0949 - mae: 5.7355
28/95 [=======>......................] - ETA: 1:03 - loss: 55.8609 - mae: 5.7218
29/95 [========>.....................] - ETA: 1:02 - loss: 56.9660 - mae: 5.7722
30/95 [========>.....................] - ETA: 1:01 - loss: 56.6087 - mae: 5.7565
31/95 [========>.....................] - ETA: 1:00 - loss: 56.3746 - mae: 5.7480
32/95 [=========>....................] - ETA: 1:00 - loss: 56.1391 - mae: 5.7413
33/95 [=========>....................] - ETA: 59s - loss: 56.2887 - mae: 5.7412 
34/95 [=========>....................] - ETA: 58s - loss: 57.0436 - mae: 5.7724
35/95 [==========>...................] - ETA: 57s - loss: 56.9672 - mae: 5.7745
36/95 [==========>...................] - ETA: 56s - loss: 57.0806 - mae: 5.7723
37/95 [==========>...................] - ETA: 55s - loss: 57.2467 - mae: 5.7813
38/95 [===========>..................] - ETA: 54s - loss: 56.8225 - mae: 5.7687
39/95 [===========>..................] - ETA: 53s - loss: 56.7564 - mae: 5.7571
40/95 [===========>..................] - ETA: 52s - loss: 56.5443 - mae: 5.7406
41/95 [===========>..................] - ETA: 51s - loss: 56.6823 - mae: 5.7460
42/95 [============>.................] - ETA: 50s - loss: 56.4591 - mae: 5.7356
43/95 [============>.................] - ETA: 49s - loss: 56.1676 - mae: 5.7202
44/95 [============>.................] - ETA: 48s - loss: 56.7244 - mae: 5.7477
45/95 [=============>................] - ETA: 47s - loss: 56.4489 - mae: 5.7291
46/95 [=============>................] - ETA: 46s - loss: 56.3164 - mae: 5.7225
47/95 [=============>................] - ETA: 45s - loss: 56.5890 - mae: 5.7399
48/95 [==============>...............] - ETA: 44s - loss: 56.7514 - mae: 5.7468
49/95 [==============>...............] - ETA: 43s - loss: 56.2878 - mae: 5.7290
50/95 [==============>...............] - ETA: 42s - loss: 56.2488 - mae: 5.7339
51/95 [===============>..............] - ETA: 41s - loss: 56.2665 - mae: 5.7481
52/95 [===============>..............] - ETA: 40s - loss: 56.3039 - mae: 5.7431
53/95 [===============>..............] - ETA: 39s - loss: 56.1316 - mae: 5.7438
54/95 [================>.............] - ETA: 38s - loss: 55.8484 - mae: 5.7242
55/95 [================>.............] - ETA: 38s - loss: 56.4406 - mae: 5.7376
56/95 [================>.............] - ETA: 37s - loss: 56.3005 - mae: 5.7285
57/95 [=================>............] - ETA: 36s - loss: 56.1159 - mae: 5.7169
58/95 [=================>............] - ETA: 35s - loss: 55.8828 - mae: 5.7069
59/95 [=================>............] - ETA: 34s - loss: 55.9122 - mae: 5.7097
60/95 [=================>............] - ETA: 33s - loss: 56.2432 - mae: 5.7279
61/95 [==================>...........] - ETA: 32s - loss: 56.3813 - mae: 5.7276
62/95 [==================>...........] - ETA: 31s - loss: 56.3136 - mae: 5.7283
63/95 [==================>...........] - ETA: 30s - loss: 56.1755 - mae: 5.7204
64/95 [===================>..........] - ETA: 29s - loss: 56.0234 - mae: 5.7169
65/95 [===================>..........] - ETA: 28s - loss: 56.0393 - mae: 5.7144
66/95 [===================>..........] - ETA: 27s - loss: 55.8034 - mae: 5.7032
67/95 [====================>.........] - ETA: 26s - loss: 56.0376 - mae: 5.7190
68/95 [====================>.........] - ETA: 25s - loss: 55.7277 - mae: 5.7055
69/95 [====================>.........] - ETA: 24s - loss: 56.0586 - mae: 5.7199
70/95 [=====================>........] - ETA: 23s - loss: 56.3734 - mae: 5.7317
71/95 [=====================>........] - ETA: 22s - loss: 56.0418 - mae: 5.7171
72/95 [=====================>........] - ETA: 21s - loss: 55.9902 - mae: 5.7175
73/95 [======================>.......] - ETA: 20s - loss: 55.7670 - mae: 5.7072
74/95 [======================>.......] - ETA: 19s - loss: 56.0894 - mae: 5.7237
75/95 [======================>.......] - ETA: 18s - loss: 55.9856 - mae: 5.7213
76/95 [=======================>......] - ETA: 18s - loss: 56.0462 - mae: 5.7230
77/95 [=======================>......] - ETA: 17s - loss: 56.0844 - mae: 5.7212
78/95 [=======================>......] - ETA: 16s - loss: 55.9701 - mae: 5.7155
79/95 [=======================>......] - ETA: 15s - loss: 56.0099 - mae: 5.7206
80/95 [========================>.....] - ETA: 14s - loss: 56.7194 - mae: 5.7488
81/95 [========================>.....] - ETA: 13s - loss: 56.5945 - mae: 5.7431
82/95 [========================>.....] - ETA: 12s - loss: 56.6804 - mae: 5.7479
83/95 [=========================>....] - ETA: 11s - loss: 56.7904 - mae: 5.7527
84/95 [=========================>....] - ETA: 10s - loss: 56.6841 - mae: 5.7465
85/95 [=========================>....] - ETA: 9s - loss: 56.6158 - mae: 5.7418 
86/95 [==========================>...] - ETA: 8s - loss: 56.5135 - mae: 5.7380
87/95 [==========================>...] - ETA: 7s - loss: 56.7297 - mae: 5.7483
88/95 [==========================>...] - ETA: 6s - loss: 57.0108 - mae: 5.7670
89/95 [===========================>..] - ETA: 5s - loss: 56.9637 - mae: 5.7649
90/95 [===========================>..] - ETA: 4s - loss: 57.0652 - mae: 5.7686
91/95 [===========================>..] - ETA: 3s - loss: 57.0887 - mae: 5.7705
92/95 [============================>.] - ETA: 2s - loss: 57.0304 - mae: 5.7689
93/95 [============================>.] - ETA: 1s - loss: 57.0306 - mae: 5.7659
94/95 [============================>.] - ETA: 0s - loss: 57.0256 - mae: 5.7653
95/95 [==============================] - 99s 1s/step - loss: 57.1357 - mae: 5.7704 - val_loss: 81.7915 - val_mae: 6.7730
Epoch 13/20

 1/95 [..............................] - ETA: 1:43 - loss: 35.1843 - mae: 4.9826
 2/95 [..............................] - ETA: 1:31 - loss: 39.1165 - mae: 5.2217
 3/95 [..............................] - ETA: 1:29 - loss: 42.2641 - mae: 5.1067
 4/95 [>.............................] - ETA: 1:25 - loss: 46.5522 - mae: 5.3122
 5/95 [>.............................] - ETA: 1:24 - loss: 44.1149 - mae: 5.2565
 6/95 [>.............................] - ETA: 1:22 - loss: 44.0905 - mae: 5.2576
 7/95 [=>............................] - ETA: 1:21 - loss: 45.5908 - mae: 5.2893
 8/95 [=>............................] - ETA: 1:20 - loss: 46.8085 - mae: 5.3232
 9/95 [=>............................] - ETA: 1:19 - loss: 48.4894 - mae: 5.4265
10/95 [==>...........................] - ETA: 1:18 - loss: 48.7366 - mae: 5.4453
11/95 [==>...........................] - ETA: 1:20 - loss: 48.7285 - mae: 5.4543
12/95 [==>...........................] - ETA: 1:19 - loss: 48.8503 - mae: 5.4377
13/95 [===>..........................] - ETA: 1:18 - loss: 49.6673 - mae: 5.4963
14/95 [===>..........................] - ETA: 1:17 - loss: 50.7022 - mae: 5.5095
15/95 [===>..........................] - ETA: 1:15 - loss: 51.9511 - mae: 5.5764
16/95 [====>.........................] - ETA: 1:14 - loss: 51.8337 - mae: 5.5495
17/95 [====>.........................] - ETA: 1:14 - loss: 50.7522 - mae: 5.4998
18/95 [====>.........................] - ETA: 1:13 - loss: 50.3323 - mae: 5.4707
19/95 [=====>........................] - ETA: 1:12 - loss: 50.4545 - mae: 5.4721
20/95 [=====>........................] - ETA: 1:11 - loss: 49.9174 - mae: 5.4521
21/95 [=====>........................] - ETA: 1:10 - loss: 50.2875 - mae: 5.4695
22/95 [=====>........................] - ETA: 1:09 - loss: 51.0824 - mae: 5.5221
23/95 [======>.......................] - ETA: 1:08 - loss: 50.6277 - mae: 5.4646
24/95 [======>.......................] - ETA: 1:07 - loss: 50.0725 - mae: 5.4236
25/95 [======>.......................] - ETA: 1:06 - loss: 49.8717 - mae: 5.4231
26/95 [=======>......................] - ETA: 1:05 - loss: 49.6030 - mae: 5.4073
27/95 [=======>......................] - ETA: 1:04 - loss: 49.9910 - mae: 5.4347
28/95 [=======>......................] - ETA: 1:03 - loss: 50.3993 - mae: 5.4455
29/95 [========>.....................] - ETA: 1:03 - loss: 50.2578 - mae: 5.4330
30/95 [========>.....................] - ETA: 1:02 - loss: 50.4389 - mae: 5.4408
31/95 [========>.....................] - ETA: 1:01 - loss: 50.6428 - mae: 5.4689
32/95 [=========>....................] - ETA: 1:00 - loss: 50.0779 - mae: 5.4454
33/95 [=========>....................] - ETA: 59s - loss: 49.9088 - mae: 5.4310 
34/95 [=========>....................] - ETA: 58s - loss: 49.9935 - mae: 5.4527
35/95 [==========>...................] - ETA: 57s - loss: 50.0343 - mae: 5.4524
36/95 [==========>...................] - ETA: 57s - loss: 49.9895 - mae: 5.4502
37/95 [==========>...................] - ETA: 56s - loss: 50.2086 - mae: 5.4711
38/95 [===========>..................] - ETA: 54s - loss: 51.2267 - mae: 5.5194
39/95 [===========>..................] - ETA: 53s - loss: 51.3136 - mae: 5.5167
40/95 [===========>..................] - ETA: 52s - loss: 50.9118 - mae: 5.4984
41/95 [===========>..................] - ETA: 51s - loss: 51.1766 - mae: 5.5167
42/95 [============>.................] - ETA: 50s - loss: 50.8508 - mae: 5.4974
43/95 [============>.................] - ETA: 49s - loss: 50.8858 - mae: 5.5045
44/95 [============>.................] - ETA: 49s - loss: 51.0100 - mae: 5.5168
45/95 [=============>................] - ETA: 48s - loss: 50.8437 - mae: 5.5125
46/95 [=============>................] - ETA: 47s - loss: 50.5240 - mae: 5.4961
47/95 [=============>................] - ETA: 46s - loss: 50.6051 - mae: 5.5026
48/95 [==============>...............] - ETA: 45s - loss: 50.5034 - mae: 5.5056
49/95 [==============>...............] - ETA: 44s - loss: 50.3156 - mae: 5.4954
50/95 [==============>...............] - ETA: 43s - loss: 50.1416 - mae: 5.4872
51/95 [===============>..............] - ETA: 42s - loss: 49.9215 - mae: 5.4658
52/95 [===============>..............] - ETA: 41s - loss: 49.8360 - mae: 5.4595
53/95 [===============>..............] - ETA: 40s - loss: 49.8088 - mae: 5.4549
54/95 [================>.............] - ETA: 39s - loss: 49.7040 - mae: 5.4436
55/95 [================>.............] - ETA: 38s - loss: 49.6673 - mae: 5.4418
56/95 [================>.............] - ETA: 37s - loss: 49.7321 - mae: 5.4483
57/95 [=================>............] - ETA: 36s - loss: 49.6396 - mae: 5.4403
58/95 [=================>............] - ETA: 35s - loss: 49.5607 - mae: 5.4390
59/95 [=================>............] - ETA: 34s - loss: 49.3089 - mae: 5.4274
60/95 [=================>............] - ETA: 33s - loss: 49.4771 - mae: 5.4400
61/95 [==================>...........] - ETA: 32s - loss: 49.4854 - mae: 5.4408
62/95 [==================>...........] - ETA: 31s - loss: 49.4523 - mae: 5.4392
63/95 [==================>...........] - ETA: 30s - loss: 49.3253 - mae: 5.4287
64/95 [===================>..........] - ETA: 29s - loss: 49.2914 - mae: 5.4287
65/95 [===================>..........] - ETA: 28s - loss: 49.2140 - mae: 5.4203
66/95 [===================>..........] - ETA: 28s - loss: 49.2026 - mae: 5.4149
67/95 [====================>.........] - ETA: 27s - loss: 49.5341 - mae: 5.4346
68/95 [====================>.........] - ETA: 26s - loss: 49.7605 - mae: 5.4503
69/95 [====================>.........] - ETA: 25s - loss: 49.8195 - mae: 5.4551
70/95 [=====================>........] - ETA: 24s - loss: 49.9290 - mae: 5.4678
71/95 [=====================>........] - ETA: 23s - loss: 49.7816 - mae: 5.4612
72/95 [=====================>........] - ETA: 22s - loss: 49.9406 - mae: 5.4756
73/95 [======================>.......] - ETA: 21s - loss: 49.8472 - mae: 5.4696
74/95 [======================>.......] - ETA: 20s - loss: 49.9233 - mae: 5.4730
75/95 [======================>.......] - ETA: 19s - loss: 49.8517 - mae: 5.4741
76/95 [=======================>......] - ETA: 18s - loss: 49.8176 - mae: 5.4658
77/95 [=======================>......] - ETA: 17s - loss: 50.1831 - mae: 5.4807
78/95 [=======================>......] - ETA: 16s - loss: 50.1000 - mae: 5.4754
79/95 [=======================>......] - ETA: 15s - loss: 50.4526 - mae: 5.4919
80/95 [========================>.....] - ETA: 14s - loss: 50.7900 - mae: 5.5016
81/95 [========================>.....] - ETA: 13s - loss: 50.9239 - mae: 5.5140
82/95 [========================>.....] - ETA: 12s - loss: 50.8149 - mae: 5.5084
83/95 [=========================>....] - ETA: 11s - loss: 50.8393 - mae: 5.5112
84/95 [=========================>....] - ETA: 10s - loss: 50.9275 - mae: 5.5175
85/95 [=========================>....] - ETA: 9s - loss: 51.0835 - mae: 5.5217 
86/95 [==========================>...] - ETA: 8s - loss: 51.1548 - mae: 5.5261
87/95 [==========================>...] - ETA: 7s - loss: 50.9876 - mae: 5.5133
88/95 [==========================>...] - ETA: 6s - loss: 50.8446 - mae: 5.5066
89/95 [===========================>..] - ETA: 5s - loss: 50.9311 - mae: 5.5099
90/95 [===========================>..] - ETA: 4s - loss: 50.8976 - mae: 5.5087
91/95 [===========================>..] - ETA: 3s - loss: 50.8257 - mae: 5.5061
92/95 [============================>.] - ETA: 2s - loss: 50.7049 - mae: 5.4973
93/95 [============================>.] - ETA: 1s - loss: 51.0350 - mae: 5.5127
94/95 [============================>.] - ETA: 0s - loss: 51.0119 - mae: 5.5137
95/95 [==============================] - 99s 1s/step - loss: 50.9690 - mae: 5.5125 - val_loss: 86.0987 - val_mae: 7.1055
Epoch 14/20

 1/95 [..............................] - ETA: 1:49 - loss: 43.2696 - mae: 5.0230
 2/95 [..............................] - ETA: 1:36 - loss: 38.1127 - mae: 4.7426
 3/95 [..............................] - ETA: 1:32 - loss: 51.6240 - mae: 5.3984
 4/95 [>.............................] - ETA: 1:29 - loss: 47.4800 - mae: 5.2266
 5/95 [>.............................] - ETA: 1:28 - loss: 46.6869 - mae: 5.2374
 6/95 [>.............................] - ETA: 1:26 - loss: 46.6583 - mae: 5.3000
 7/95 [=>............................] - ETA: 1:24 - loss: 43.6996 - mae: 5.1301
 8/95 [=>............................] - ETA: 1:22 - loss: 42.3556 - mae: 5.0328
 9/95 [=>............................] - ETA: 1:21 - loss: 41.0088 - mae: 4.9629
10/95 [==>...........................] - ETA: 1:21 - loss: 40.5436 - mae: 4.9310
11/95 [==>...........................] - ETA: 1:19 - loss: 42.5439 - mae: 5.0213
12/95 [==>...........................] - ETA: 1:18 - loss: 42.9099 - mae: 5.0427
13/95 [===>..........................] - ETA: 1:18 - loss: 43.1211 - mae: 5.0414
14/95 [===>..........................] - ETA: 1:16 - loss: 44.7439 - mae: 5.0591
15/95 [===>..........................] - ETA: 1:15 - loss: 45.7332 - mae: 5.0959
16/95 [====>.........................] - ETA: 1:14 - loss: 46.9006 - mae: 5.1532
17/95 [====>.........................] - ETA: 1:13 - loss: 45.8440 - mae: 5.1027
18/95 [====>.........................] - ETA: 1:12 - loss: 45.8897 - mae: 5.1273
19/95 [=====>........................] - ETA: 1:12 - loss: 45.0665 - mae: 5.0889
20/95 [=====>........................] - ETA: 1:10 - loss: 46.3398 - mae: 5.1565
21/95 [=====>........................] - ETA: 1:10 - loss: 46.1129 - mae: 5.1548
22/95 [=====>........................] - ETA: 1:09 - loss: 46.3104 - mae: 5.1718
23/95 [======>.......................] - ETA: 1:08 - loss: 47.0408 - mae: 5.2157
24/95 [======>.......................] - ETA: 1:06 - loss: 47.1869 - mae: 5.2306
25/95 [======>.......................] - ETA: 1:05 - loss: 48.2810 - mae: 5.2712
26/95 [=======>......................] - ETA: 1:04 - loss: 48.3973 - mae: 5.2656
27/95 [=======>......................] - ETA: 1:04 - loss: 48.0269 - mae: 5.2457
28/95 [=======>......................] - ETA: 1:03 - loss: 47.5737 - mae: 5.2303
29/95 [========>.....................] - ETA: 1:02 - loss: 47.3597 - mae: 5.2232
30/95 [========>.....................] - ETA: 1:01 - loss: 47.5467 - mae: 5.2360
31/95 [========>.....................] - ETA: 1:00 - loss: 47.8223 - mae: 5.2593
32/95 [=========>....................] - ETA: 59s - loss: 47.8249 - mae: 5.2694 
33/95 [=========>....................] - ETA: 58s - loss: 47.6527 - mae: 5.2751
34/95 [=========>....................] - ETA: 57s - loss: 47.6158 - mae: 5.2726
35/95 [==========>...................] - ETA: 56s - loss: 47.0776 - mae: 5.2427
36/95 [==========>...................] - ETA: 55s - loss: 46.8292 - mae: 5.2334
37/95 [==========>...................] - ETA: 54s - loss: 46.9634 - mae: 5.2359
38/95 [===========>..................] - ETA: 53s - loss: 46.9123 - mae: 5.2354
39/95 [===========>..................] - ETA: 52s - loss: 46.8642 - mae: 5.2300
40/95 [===========>..................] - ETA: 51s - loss: 46.6779 - mae: 5.2129
41/95 [===========>..................] - ETA: 50s - loss: 46.8181 - mae: 5.2246
42/95 [============>.................] - ETA: 50s - loss: 47.0209 - mae: 5.2440
43/95 [============>.................] - ETA: 49s - loss: 46.8598 - mae: 5.2314
44/95 [============>.................] - ETA: 48s - loss: 47.0855 - mae: 5.2390
45/95 [=============>................] - ETA: 47s - loss: 46.8480 - mae: 5.2174
46/95 [=============>................] - ETA: 46s - loss: 46.9667 - mae: 5.2276
47/95 [=============>................] - ETA: 45s - loss: 46.7987 - mae: 5.2167
48/95 [==============>...............] - ETA: 44s - loss: 46.8541 - mae: 5.2166
49/95 [==============>...............] - ETA: 43s - loss: 46.7545 - mae: 5.2140
50/95 [==============>...............] - ETA: 43s - loss: 46.7139 - mae: 5.2144
51/95 [===============>..............] - ETA: 42s - loss: 46.8405 - mae: 5.2281
52/95 [===============>..............] - ETA: 41s - loss: 46.7805 - mae: 5.2261
53/95 [===============>..............] - ETA: 40s - loss: 46.8097 - mae: 5.2307
54/95 [================>.............] - ETA: 39s - loss: 46.6424 - mae: 5.2210
55/95 [================>.............] - ETA: 38s - loss: 46.5637 - mae: 5.2140
56/95 [================>.............] - ETA: 37s - loss: 46.8253 - mae: 5.2271
57/95 [=================>............] - ETA: 36s - loss: 46.7325 - mae: 5.2277
58/95 [=================>............] - ETA: 35s - loss: 46.9829 - mae: 5.2454
59/95 [=================>............] - ETA: 34s - loss: 47.1000 - mae: 5.2521
60/95 [=================>............] - ETA: 33s - loss: 46.9906 - mae: 5.2417
61/95 [==================>...........] - ETA: 32s - loss: 47.1229 - mae: 5.2441
62/95 [==================>...........] - ETA: 31s - loss: 47.3271 - mae: 5.2615
63/95 [==================>...........] - ETA: 30s - loss: 47.4257 - mae: 5.2679
64/95 [===================>..........] - ETA: 29s - loss: 47.2204 - mae: 5.2502
65/95 [===================>..........] - ETA: 28s - loss: 47.1621 - mae: 5.2505
66/95 [===================>..........] - ETA: 27s - loss: 47.0473 - mae: 5.2463
67/95 [====================>.........] - ETA: 26s - loss: 47.0716 - mae: 5.2504
68/95 [====================>.........] - ETA: 25s - loss: 47.5936 - mae: 5.2650
69/95 [====================>.........] - ETA: 24s - loss: 47.7754 - mae: 5.2718
70/95 [=====================>........] - ETA: 23s - loss: 47.8367 - mae: 5.2723
71/95 [=====================>........] - ETA: 23s - loss: 47.8721 - mae: 5.2788
72/95 [=====================>........] - ETA: 22s - loss: 47.7951 - mae: 5.2753
73/95 [======================>.......] - ETA: 21s - loss: 47.7413 - mae: 5.2744
74/95 [======================>.......] - ETA: 20s - loss: 47.7672 - mae: 5.2778
75/95 [======================>.......] - ETA: 19s - loss: 47.9619 - mae: 5.2869
76/95 [=======================>......] - ETA: 18s - loss: 47.9670 - mae: 5.2844
77/95 [=======================>......] - ETA: 17s - loss: 48.0325 - mae: 5.2909
78/95 [=======================>......] - ETA: 16s - loss: 48.0662 - mae: 5.2968
79/95 [=======================>......] - ETA: 15s - loss: 47.9669 - mae: 5.2913
80/95 [========================>.....] - ETA: 14s - loss: 47.8811 - mae: 5.2911
81/95 [========================>.....] - ETA: 13s - loss: 47.7232 - mae: 5.2854
82/95 [========================>.....] - ETA: 12s - loss: 47.5399 - mae: 5.2775
83/95 [=========================>....] - ETA: 11s - loss: 47.7028 - mae: 5.2893
84/95 [=========================>....] - ETA: 10s - loss: 47.7892 - mae: 5.2948
85/95 [=========================>....] - ETA: 9s - loss: 47.8642 - mae: 5.3003 
86/95 [==========================>...] - ETA: 8s - loss: 47.9004 - mae: 5.3038
87/95 [==========================>...] - ETA: 7s - loss: 48.2201 - mae: 5.3185
88/95 [==========================>...] - ETA: 6s - loss: 48.1732 - mae: 5.3183
89/95 [===========================>..] - ETA: 5s - loss: 48.1127 - mae: 5.3191
90/95 [===========================>..] - ETA: 4s - loss: 48.2105 - mae: 5.3198
91/95 [===========================>..] - ETA: 3s - loss: 48.3355 - mae: 5.3280
92/95 [============================>.] - ETA: 2s - loss: 48.3163 - mae: 5.3283
93/95 [============================>.] - ETA: 1s - loss: 48.4277 - mae: 5.3358
94/95 [============================>.] - ETA: 0s - loss: 48.5545 - mae: 5.3406
95/95 [==============================] - 99s 1s/step - loss: 48.4171 - mae: 5.3357 - val_loss: 86.8563 - val_mae: 6.7377
Epoch 15/20

 1/95 [..............................] - ETA: 1:41 - loss: 49.9152 - mae: 5.6642
 2/95 [..............................] - ETA: 1:32 - loss: 45.3611 - mae: 5.2910
 3/95 [..............................] - ETA: 1:33 - loss: 48.0845 - mae: 5.1566
 4/95 [>.............................] - ETA: 1:32 - loss: 45.1985 - mae: 5.0130
 5/95 [>.............................] - ETA: 1:35 - loss: 48.1252 - mae: 5.1690
 6/95 [>.............................] - ETA: 1:32 - loss: 46.1417 - mae: 5.0350
 7/95 [=>............................] - ETA: 1:29 - loss: 47.9423 - mae: 5.1638
 8/95 [=>............................] - ETA: 1:28 - loss: 47.1894 - mae: 5.1397
 9/95 [=>............................] - ETA: 1:26 - loss: 45.9827 - mae: 5.0658
10/95 [==>...........................] - ETA: 1:25 - loss: 46.9239 - mae: 5.1147
11/95 [==>...........................] - ETA: 1:24 - loss: 47.7346 - mae: 5.1992
12/95 [==>...........................] - ETA: 1:22 - loss: 46.6168 - mae: 5.1372
13/95 [===>..........................] - ETA: 1:21 - loss: 46.1836 - mae: 5.1193
14/95 [===>..........................] - ETA: 1:20 - loss: 45.8173 - mae: 5.0925
15/95 [===>..........................] - ETA: 1:18 - loss: 46.0986 - mae: 5.1097
16/95 [====>.........................] - ETA: 1:17 - loss: 45.8262 - mae: 5.0960
17/95 [====>.........................] - ETA: 1:16 - loss: 46.1164 - mae: 5.1264
18/95 [====>.........................] - ETA: 1:15 - loss: 45.7011 - mae: 5.1046
19/95 [=====>........................] - ETA: 1:14 - loss: 47.2010 - mae: 5.1941
20/95 [=====>........................] - ETA: 1:13 - loss: 48.7746 - mae: 5.2654
21/95 [=====>........................] - ETA: 1:12 - loss: 48.9932 - mae: 5.3031
22/95 [=====>........................] - ETA: 1:10 - loss: 49.0617 - mae: 5.3083
23/95 [======>.......................] - ETA: 1:10 - loss: 49.2738 - mae: 5.3317
24/95 [======>.......................] - ETA: 1:09 - loss: 48.8500 - mae: 5.3272
25/95 [======>.......................] - ETA: 1:07 - loss: 48.9460 - mae: 5.3394
26/95 [=======>......................] - ETA: 1:07 - loss: 48.7648 - mae: 5.3454
27/95 [=======>......................] - ETA: 1:06 - loss: 48.7080 - mae: 5.3424
28/95 [=======>......................] - ETA: 1:05 - loss: 48.5169 - mae: 5.3311
29/95 [========>.....................] - ETA: 1:04 - loss: 47.8575 - mae: 5.2920
30/95 [========>.....................] - ETA: 1:03 - loss: 47.3187 - mae: 5.2624
31/95 [========>.....................] - ETA: 1:02 - loss: 47.4069 - mae: 5.2721
32/95 [=========>....................] - ETA: 1:01 - loss: 47.4174 - mae: 5.2781
33/95 [=========>....................] - ETA: 1:00 - loss: 48.2549 - mae: 5.3081
34/95 [=========>....................] - ETA: 58s - loss: 48.1546 - mae: 5.2992 
35/95 [==========>...................] - ETA: 57s - loss: 47.7260 - mae: 5.2748
36/95 [==========>...................] - ETA: 56s - loss: 47.3905 - mae: 5.2528
37/95 [==========>...................] - ETA: 55s - loss: 47.2993 - mae: 5.2499
38/95 [===========>..................] - ETA: 54s - loss: 47.2361 - mae: 5.2524
39/95 [===========>..................] - ETA: 53s - loss: 46.8974 - mae: 5.2375
40/95 [===========>..................] - ETA: 52s - loss: 47.0280 - mae: 5.2448
41/95 [===========>..................] - ETA: 51s - loss: 47.0753 - mae: 5.2422
42/95 [============>.................] - ETA: 50s - loss: 46.8958 - mae: 5.2433
43/95 [============>.................] - ETA: 49s - loss: 46.7849 - mae: 5.2472
44/95 [============>.................] - ETA: 49s - loss: 47.1919 - mae: 5.2620
45/95 [=============>................] - ETA: 48s - loss: 47.1906 - mae: 5.2518
46/95 [=============>................] - ETA: 47s - loss: 46.9432 - mae: 5.2365
47/95 [=============>................] - ETA: 45s - loss: 46.6347 - mae: 5.2188
48/95 [==============>...............] - ETA: 44s - loss: 46.5414 - mae: 5.2057
49/95 [==============>...............] - ETA: 43s - loss: 46.7892 - mae: 5.2176
50/95 [==============>...............] - ETA: 43s - loss: 46.8721 - mae: 5.2230
51/95 [===============>..............] - ETA: 42s - loss: 46.8735 - mae: 5.2217
52/95 [===============>..............] - ETA: 41s - loss: 46.9976 - mae: 5.2268
53/95 [===============>..............] - ETA: 40s - loss: 46.9384 - mae: 5.2280
54/95 [================>.............] - ETA: 39s - loss: 47.1452 - mae: 5.2396
55/95 [================>.............] - ETA: 38s - loss: 46.8706 - mae: 5.2268
56/95 [================>.............] - ETA: 37s - loss: 46.8512 - mae: 5.2270
57/95 [=================>............] - ETA: 36s - loss: 46.8546 - mae: 5.2277
58/95 [=================>............] - ETA: 35s - loss: 47.0064 - mae: 5.2294
59/95 [=================>............] - ETA: 34s - loss: 46.8243 - mae: 5.2195
60/95 [=================>............] - ETA: 33s - loss: 46.9548 - mae: 5.2237
61/95 [==================>...........] - ETA: 32s - loss: 46.7362 - mae: 5.2113
62/95 [==================>...........] - ETA: 31s - loss: 47.0238 - mae: 5.2331
63/95 [==================>...........] - ETA: 30s - loss: 46.8757 - mae: 5.2294
64/95 [===================>..........] - ETA: 29s - loss: 46.7845 - mae: 5.2204
65/95 [===================>..........] - ETA: 28s - loss: 46.9416 - mae: 5.2252
66/95 [===================>..........] - ETA: 27s - loss: 46.8093 - mae: 5.2202
67/95 [====================>.........] - ETA: 26s - loss: 46.8172 - mae: 5.2252
68/95 [====================>.........] - ETA: 25s - loss: 46.8209 - mae: 5.2224
69/95 [====================>.........] - ETA: 24s - loss: 46.6485 - mae: 5.2181
70/95 [=====================>........] - ETA: 23s - loss: 46.7508 - mae: 5.2227
71/95 [=====================>........] - ETA: 22s - loss: 46.8560 - mae: 5.2338
72/95 [=====================>........] - ETA: 21s - loss: 46.9429 - mae: 5.2379
73/95 [======================>.......] - ETA: 20s - loss: 46.9889 - mae: 5.2421
74/95 [======================>.......] - ETA: 19s - loss: 46.8805 - mae: 5.2382
75/95 [======================>.......] - ETA: 19s - loss: 47.0815 - mae: 5.2437
76/95 [=======================>......] - ETA: 18s - loss: 47.0922 - mae: 5.2491
77/95 [=======================>......] - ETA: 17s - loss: 47.0208 - mae: 5.2468
78/95 [=======================>......] - ETA: 16s - loss: 46.9652 - mae: 5.2483
79/95 [=======================>......] - ETA: 15s - loss: 46.8999 - mae: 5.2460
80/95 [========================>.....] - ETA: 14s - loss: 47.1838 - mae: 5.2584
81/95 [========================>.....] - ETA: 13s - loss: 47.1602 - mae: 5.2595
82/95 [========================>.....] - ETA: 12s - loss: 47.2557 - mae: 5.2686
83/95 [=========================>....] - ETA: 11s - loss: 47.4814 - mae: 5.2820
84/95 [=========================>....] - ETA: 10s - loss: 47.2769 - mae: 5.2699
85/95 [=========================>....] - ETA: 9s - loss: 47.3018 - mae: 5.2754 
86/95 [==========================>...] - ETA: 8s - loss: 47.3641 - mae: 5.2805
87/95 [==========================>...] - ETA: 7s - loss: 47.3385 - mae: 5.2802
88/95 [==========================>...] - ETA: 6s - loss: 47.2161 - mae: 5.2722
89/95 [===========================>..] - ETA: 5s - loss: 47.2095 - mae: 5.2725
90/95 [===========================>..] - ETA: 4s - loss: 47.1016 - mae: 5.2654
91/95 [===========================>..] - ETA: 3s - loss: 47.1911 - mae: 5.2678
92/95 [============================>.] - ETA: 2s - loss: 47.2253 - mae: 5.2715
93/95 [============================>.] - ETA: 1s - loss: 47.5274 - mae: 5.2841
94/95 [============================>.] - ETA: 0s - loss: 47.4911 - mae: 5.2851
95/95 [==============================] - 98s 1s/step - loss: 47.6560 - mae: 5.2913 - val_loss: 106.3282 - val_mae: 7.3724
Epoch 16/20

 1/95 [..............................] - ETA: 1:39 - loss: 37.9241 - mae: 4.5539
 2/95 [..............................] - ETA: 1:41 - loss: 44.7999 - mae: 4.9000
 3/95 [..............................] - ETA: 1:35 - loss: 42.4532 - mae: 5.0148
 4/95 [>.............................] - ETA: 1:34 - loss: 44.6107 - mae: 5.2186
 5/95 [>.............................] - ETA: 1:37 - loss: 47.2393 - mae: 5.3401
 6/95 [>.............................] - ETA: 1:34 - loss: 47.2336 - mae: 5.3405
 7/95 [=>............................] - ETA: 1:33 - loss: 45.9980 - mae: 5.2790
 8/95 [=>............................] - ETA: 1:30 - loss: 45.6481 - mae: 5.2586
 9/95 [=>............................] - ETA: 1:28 - loss: 48.7956 - mae: 5.3823
10/95 [==>...........................] - ETA: 1:26 - loss: 47.4002 - mae: 5.3274
11/95 [==>...........................] - ETA: 1:24 - loss: 48.9765 - mae: 5.4413
12/95 [==>...........................] - ETA: 1:22 - loss: 50.5197 - mae: 5.5420
13/95 [===>..........................] - ETA: 1:21 - loss: 51.1081 - mae: 5.5201
14/95 [===>..........................] - ETA: 1:19 - loss: 50.4840 - mae: 5.4619
15/95 [===>..........................] - ETA: 1:18 - loss: 50.9910 - mae: 5.5121
16/95 [====>.........................] - ETA: 1:17 - loss: 50.6946 - mae: 5.5146
17/95 [====>.........................] - ETA: 1:16 - loss: 51.1415 - mae: 5.5192
18/95 [====>.........................] - ETA: 1:15 - loss: 50.7798 - mae: 5.5056
19/95 [=====>........................] - ETA: 1:14 - loss: 50.2234 - mae: 5.4529
20/95 [=====>........................] - ETA: 1:13 - loss: 49.7672 - mae: 5.4350
21/95 [=====>........................] - ETA: 1:12 - loss: 49.6741 - mae: 5.4184
22/95 [=====>........................] - ETA: 1:11 - loss: 50.1863 - mae: 5.4416
23/95 [======>.......................] - ETA: 1:10 - loss: 50.0848 - mae: 5.4423
24/95 [======>.......................] - ETA: 1:09 - loss: 49.3989 - mae: 5.4144
25/95 [======>.......................] - ETA: 1:08 - loss: 48.6844 - mae: 5.3635
26/95 [=======>......................] - ETA: 1:07 - loss: 48.7664 - mae: 5.3896
27/95 [=======>......................] - ETA: 1:06 - loss: 48.4086 - mae: 5.3599
28/95 [=======>......................] - ETA: 1:05 - loss: 48.3340 - mae: 5.3610
29/95 [========>.....................] - ETA: 1:04 - loss: 48.1035 - mae: 5.3563
30/95 [========>.....................] - ETA: 1:03 - loss: 47.9925 - mae: 5.3527
31/95 [========>.....................] - ETA: 1:02 - loss: 47.5995 - mae: 5.3130
32/95 [=========>....................] - ETA: 1:01 - loss: 47.6575 - mae: 5.3115
33/95 [=========>....................] - ETA: 1:00 - loss: 47.2850 - mae: 5.2871
34/95 [=========>....................] - ETA: 59s - loss: 47.0964 - mae: 5.2765 
35/95 [==========>...................] - ETA: 58s - loss: 46.6038 - mae: 5.2539
36/95 [==========>...................] - ETA: 57s - loss: 46.5334 - mae: 5.2487
37/95 [==========>...................] - ETA: 56s - loss: 46.4980 - mae: 5.2568
38/95 [===========>..................] - ETA: 55s - loss: 46.7864 - mae: 5.2691
39/95 [===========>..................] - ETA: 54s - loss: 47.0674 - mae: 5.2920
40/95 [===========>..................] - ETA: 53s - loss: 46.8015 - mae: 5.2754
41/95 [===========>..................] - ETA: 52s - loss: 47.0184 - mae: 5.2859
42/95 [============>.................] - ETA: 51s - loss: 46.6976 - mae: 5.2606
43/95 [============>.................] - ETA: 50s - loss: 46.6939 - mae: 5.2567
44/95 [============>.................] - ETA: 49s - loss: 46.6997 - mae: 5.2559
45/95 [=============>................] - ETA: 48s - loss: 46.5057 - mae: 5.2454
46/95 [=============>................] - ETA: 47s - loss: 46.6270 - mae: 5.2450
47/95 [=============>................] - ETA: 46s - loss: 46.3800 - mae: 5.2336
48/95 [==============>...............] - ETA: 45s - loss: 46.8175 - mae: 5.2531
49/95 [==============>...............] - ETA: 44s - loss: 46.7278 - mae: 5.2511
50/95 [==============>...............] - ETA: 43s - loss: 46.8036 - mae: 5.2455
51/95 [===============>..............] - ETA: 42s - loss: 47.3103 - mae: 5.2742
52/95 [===============>..............] - ETA: 41s - loss: 47.1358 - mae: 5.2625
53/95 [===============>..............] - ETA: 40s - loss: 47.2719 - mae: 5.2667
54/95 [================>.............] - ETA: 39s - loss: 47.0156 - mae: 5.2546
55/95 [================>.............] - ETA: 38s - loss: 46.8448 - mae: 5.2410
56/95 [================>.............] - ETA: 37s - loss: 46.7710 - mae: 5.2363
57/95 [=================>............] - ETA: 36s - loss: 46.7016 - mae: 5.2314
58/95 [=================>............] - ETA: 35s - loss: 46.6548 - mae: 5.2251
59/95 [=================>............] - ETA: 34s - loss: 46.8446 - mae: 5.2399
60/95 [=================>............] - ETA: 33s - loss: 46.6877 - mae: 5.2306
61/95 [==================>...........] - ETA: 32s - loss: 46.6210 - mae: 5.2314
62/95 [==================>...........] - ETA: 31s - loss: 46.5866 - mae: 5.2263
63/95 [==================>...........] - ETA: 30s - loss: 46.4414 - mae: 5.2112
64/95 [===================>..........] - ETA: 29s - loss: 46.4272 - mae: 5.2134
65/95 [===================>..........] - ETA: 28s - loss: 46.3517 - mae: 5.2121
66/95 [===================>..........] - ETA: 27s - loss: 46.4362 - mae: 5.2096
67/95 [====================>.........] - ETA: 26s - loss: 46.4614 - mae: 5.2118
68/95 [====================>.........] - ETA: 26s - loss: 46.4304 - mae: 5.2088
69/95 [====================>.........] - ETA: 25s - loss: 46.2487 - mae: 5.1941
70/95 [=====================>........] - ETA: 24s - loss: 46.4764 - mae: 5.1989
71/95 [=====================>........] - ETA: 23s - loss: 46.2993 - mae: 5.1896
72/95 [=====================>........] - ETA: 22s - loss: 46.1884 - mae: 5.1907
73/95 [======================>.......] - ETA: 21s - loss: 46.4025 - mae: 5.2107
74/95 [======================>.......] - ETA: 20s - loss: 46.4992 - mae: 5.2149
75/95 [======================>.......] - ETA: 19s - loss: 46.2917 - mae: 5.2076
76/95 [=======================>......] - ETA: 18s - loss: 46.3406 - mae: 5.2085
77/95 [=======================>......] - ETA: 17s - loss: 46.1120 - mae: 5.1965
78/95 [=======================>......] - ETA: 16s - loss: 46.2506 - mae: 5.2035
79/95 [=======================>......] - ETA: 15s - loss: 46.3266 - mae: 5.2099
80/95 [========================>.....] - ETA: 14s - loss: 46.1859 - mae: 5.2032
81/95 [========================>.....] - ETA: 13s - loss: 45.9618 - mae: 5.1922
82/95 [========================>.....] - ETA: 12s - loss: 46.0083 - mae: 5.1946
83/95 [=========================>....] - ETA: 11s - loss: 46.0365 - mae: 5.1947
84/95 [=========================>....] - ETA: 10s - loss: 46.2444 - mae: 5.2064
85/95 [=========================>....] - ETA: 9s - loss: 46.3564 - mae: 5.2174 
86/95 [==========================>...] - ETA: 8s - loss: 46.4332 - mae: 5.2265
87/95 [==========================>...] - ETA: 7s - loss: 46.5494 - mae: 5.2315
88/95 [==========================>...] - ETA: 6s - loss: 46.5157 - mae: 5.2331
89/95 [===========================>..] - ETA: 5s - loss: 46.2913 - mae: 5.2195
90/95 [===========================>..] - ETA: 4s - loss: 46.3968 - mae: 5.2202
91/95 [===========================>..] - ETA: 3s - loss: 46.2286 - mae: 5.2109
92/95 [============================>.] - ETA: 2s - loss: 46.1987 - mae: 5.2046
93/95 [============================>.] - ETA: 1s - loss: 46.2716 - mae: 5.2087
94/95 [============================>.] - ETA: 0s - loss: 46.1547 - mae: 5.2012
95/95 [==============================] - 100s 1s/step - loss: 46.0504 - mae: 5.1962 - val_loss: 77.4905 - val_mae: 6.4158
Epoch 17/20

 1/95 [..............................] - ETA: 1:55 - loss: 54.7129 - mae: 5.6080
 2/95 [..............................] - ETA: 1:41 - loss: 40.2881 - mae: 4.7947
 3/95 [..............................] - ETA: 1:34 - loss: 36.3617 - mae: 4.6756
 4/95 [>.............................] - ETA: 1:34 - loss: 38.9803 - mae: 4.7423
 5/95 [>.............................] - ETA: 1:32 - loss: 37.9428 - mae: 4.7558
 6/95 [>.............................] - ETA: 1:30 - loss: 38.0350 - mae: 4.8033
 7/95 [=>............................] - ETA: 1:27 - loss: 38.8591 - mae: 4.8621
 8/95 [=>............................] - ETA: 1:27 - loss: 39.1373 - mae: 4.8909
 9/95 [=>............................] - ETA: 1:25 - loss: 39.8164 - mae: 4.9297
10/95 [==>...........................] - ETA: 1:24 - loss: 39.5229 - mae: 4.8937
11/95 [==>...........................] - ETA: 1:22 - loss: 39.4009 - mae: 4.8776
12/95 [==>...........................] - ETA: 1:21 - loss: 40.0671 - mae: 4.8957
13/95 [===>..........................] - ETA: 1:20 - loss: 39.3238 - mae: 4.8578
14/95 [===>..........................] - ETA: 1:19 - loss: 38.9831 - mae: 4.8397
15/95 [===>..........................] - ETA: 1:18 - loss: 38.9926 - mae: 4.8286
16/95 [====>.........................] - ETA: 1:17 - loss: 39.0827 - mae: 4.8078
17/95 [====>.........................] - ETA: 1:15 - loss: 39.3666 - mae: 4.8291
18/95 [====>.........................] - ETA: 1:14 - loss: 39.4895 - mae: 4.8511
19/95 [=====>........................] - ETA: 1:13 - loss: 39.4098 - mae: 4.8545
20/95 [=====>........................] - ETA: 1:12 - loss: 39.4064 - mae: 4.8638
21/95 [=====>........................] - ETA: 1:11 - loss: 39.3893 - mae: 4.8593
22/95 [=====>........................] - ETA: 1:10 - loss: 39.4874 - mae: 4.8587
23/95 [======>.......................] - ETA: 1:09 - loss: 39.7242 - mae: 4.8694
24/95 [======>.......................] - ETA: 1:07 - loss: 39.3967 - mae: 4.8505
25/95 [======>.......................] - ETA: 1:07 - loss: 39.2079 - mae: 4.8473
26/95 [=======>......................] - ETA: 1:06 - loss: 39.9988 - mae: 4.8984
27/95 [=======>......................] - ETA: 1:05 - loss: 41.1556 - mae: 4.9449
28/95 [=======>......................] - ETA: 1:04 - loss: 40.8054 - mae: 4.9180
29/95 [========>.....................] - ETA: 1:03 - loss: 41.0291 - mae: 4.9321
30/95 [========>.....................] - ETA: 1:02 - loss: 40.7721 - mae: 4.9123
31/95 [========>.....................] - ETA: 1:01 - loss: 41.2827 - mae: 4.9444
32/95 [=========>....................] - ETA: 1:00 - loss: 41.1358 - mae: 4.9473
33/95 [=========>....................] - ETA: 59s - loss: 41.4210 - mae: 4.9687 
34/95 [=========>....................] - ETA: 58s - loss: 41.1995 - mae: 4.9536
35/95 [==========>...................] - ETA: 57s - loss: 41.0050 - mae: 4.9468
36/95 [==========>...................] - ETA: 56s - loss: 41.1018 - mae: 4.9579
37/95 [==========>...................] - ETA: 55s - loss: 41.6079 - mae: 4.9928
38/95 [===========>..................] - ETA: 54s - loss: 41.0302 - mae: 4.9570
39/95 [===========>..................] - ETA: 53s - loss: 41.2903 - mae: 4.9785
40/95 [===========>..................] - ETA: 53s - loss: 41.5814 - mae: 4.9830
41/95 [===========>..................] - ETA: 52s - loss: 41.7306 - mae: 4.9949
42/95 [============>.................] - ETA: 51s - loss: 41.7948 - mae: 5.0064
43/95 [============>.................] - ETA: 50s - loss: 41.6894 - mae: 5.0008
44/95 [============>.................] - ETA: 49s - loss: 41.6899 - mae: 5.0034
45/95 [=============>................] - ETA: 48s - loss: 41.7881 - mae: 5.0153
46/95 [=============>................] - ETA: 47s - loss: 41.7481 - mae: 5.0140
47/95 [=============>................] - ETA: 46s - loss: 41.9649 - mae: 5.0270
48/95 [==============>...............] - ETA: 44s - loss: 42.6539 - mae: 5.0593
49/95 [==============>...............] - ETA: 43s - loss: 42.8119 - mae: 5.0626
50/95 [==============>...............] - ETA: 42s - loss: 42.5361 - mae: 5.0419
51/95 [===============>..............] - ETA: 42s - loss: 42.5207 - mae: 5.0449
52/95 [===============>..............] - ETA: 41s - loss: 43.3221 - mae: 5.0857
53/95 [===============>..............] - ETA: 40s - loss: 43.1514 - mae: 5.0784
54/95 [================>.............] - ETA: 39s - loss: 43.0976 - mae: 5.0673
55/95 [================>.............] - ETA: 38s - loss: 43.1953 - mae: 5.0728
56/95 [================>.............] - ETA: 37s - loss: 43.2621 - mae: 5.0743
57/95 [=================>............] - ETA: 36s - loss: 43.1211 - mae: 5.0646
58/95 [=================>............] - ETA: 35s - loss: 43.0854 - mae: 5.0561
59/95 [=================>............] - ETA: 34s - loss: 43.2919 - mae: 5.0681
60/95 [=================>............] - ETA: 33s - loss: 43.6835 - mae: 5.0886
61/95 [==================>...........] - ETA: 32s - loss: 44.0460 - mae: 5.1102
62/95 [==================>...........] - ETA: 31s - loss: 43.9226 - mae: 5.1059
63/95 [==================>...........] - ETA: 30s - loss: 43.9068 - mae: 5.0997
64/95 [===================>..........] - ETA: 29s - loss: 43.8682 - mae: 5.0942
65/95 [===================>..........] - ETA: 28s - loss: 44.3177 - mae: 5.1188
66/95 [===================>..........] - ETA: 27s - loss: 45.1198 - mae: 5.1396
67/95 [====================>.........] - ETA: 26s - loss: 45.0983 - mae: 5.1368
68/95 [====================>.........] - ETA: 25s - loss: 44.9147 - mae: 5.1291
69/95 [====================>.........] - ETA: 24s - loss: 45.0710 - mae: 5.1388
70/95 [=====================>........] - ETA: 23s - loss: 44.9380 - mae: 5.1361
71/95 [=====================>........] - ETA: 22s - loss: 44.7489 - mae: 5.1286
72/95 [=====================>........] - ETA: 21s - loss: 44.8623 - mae: 5.1322
73/95 [======================>.......] - ETA: 21s - loss: 44.9205 - mae: 5.1376
74/95 [======================>.......] - ETA: 20s - loss: 45.3091 - mae: 5.1503
75/95 [======================>.......] - ETA: 19s - loss: 45.4485 - mae: 5.1565
76/95 [=======================>......] - ETA: 18s - loss: 45.4398 - mae: 5.1613
77/95 [=======================>......] - ETA: 17s - loss: 45.3646 - mae: 5.1579
78/95 [=======================>......] - ETA: 16s - loss: 45.4680 - mae: 5.1641
79/95 [=======================>......] - ETA: 15s - loss: 45.7637 - mae: 5.1795
80/95 [========================>.....] - ETA: 14s - loss: 45.7371 - mae: 5.1792
81/95 [========================>.....] - ETA: 13s - loss: 45.6235 - mae: 5.1770
82/95 [========================>.....] - ETA: 12s - loss: 45.8902 - mae: 5.1925
83/95 [=========================>....] - ETA: 11s - loss: 45.8703 - mae: 5.1926
84/95 [=========================>....] - ETA: 10s - loss: 46.0380 - mae: 5.2018
85/95 [=========================>....] - ETA: 9s - loss: 46.2250 - mae: 5.2067 
86/95 [==========================>...] - ETA: 8s - loss: 46.3320 - mae: 5.2174
87/95 [==========================>...] - ETA: 7s - loss: 46.1116 - mae: 5.2043
88/95 [==========================>...] - ETA: 6s - loss: 46.1181 - mae: 5.2036
89/95 [===========================>..] - ETA: 5s - loss: 46.1444 - mae: 5.2093
90/95 [===========================>..] - ETA: 4s - loss: 46.3198 - mae: 5.2210
91/95 [===========================>..] - ETA: 3s - loss: 46.2453 - mae: 5.2179
92/95 [============================>.] - ETA: 2s - loss: 46.2156 - mae: 5.2163
93/95 [============================>.] - ETA: 1s - loss: 46.3195 - mae: 5.2219
94/95 [============================>.] - ETA: 0s - loss: 46.2329 - mae: 5.2155
95/95 [==============================] - 99s 1s/step - loss: 46.3003 - mae: 5.2158 - val_loss: 80.9886 - val_mae: 6.5082
Epoch 18/20

 1/95 [..............................] - ETA: 1:43 - loss: 27.3178 - mae: 4.0490
 2/95 [..............................] - ETA: 1:32 - loss: 27.8639 - mae: 4.0813
 3/95 [..............................] - ETA: 1:28 - loss: 33.2769 - mae: 4.5352
 4/95 [>.............................] - ETA: 1:26 - loss: 35.2893 - mae: 4.4702
 5/95 [>.............................] - ETA: 1:25 - loss: 38.1123 - mae: 4.6590
 6/95 [>.............................] - ETA: 1:25 - loss: 38.8166 - mae: 4.7812
 7/95 [=>............................] - ETA: 1:23 - loss: 38.8614 - mae: 4.7877
 8/95 [=>............................] - ETA: 1:23 - loss: 38.1286 - mae: 4.7584
 9/95 [=>............................] - ETA: 1:21 - loss: 39.4813 - mae: 4.8462
10/95 [==>...........................] - ETA: 1:20 - loss: 38.8284 - mae: 4.8071
11/95 [==>...........................] - ETA: 1:19 - loss: 39.9317 - mae: 4.8811
12/95 [==>...........................] - ETA: 1:18 - loss: 39.6824 - mae: 4.8506
13/95 [===>..........................] - ETA: 1:17 - loss: 39.5460 - mae: 4.8323
14/95 [===>..........................] - ETA: 1:16 - loss: 39.2987 - mae: 4.8296
15/95 [===>..........................] - ETA: 1:15 - loss: 39.6445 - mae: 4.8508
16/95 [====>.........................] - ETA: 1:14 - loss: 39.3598 - mae: 4.8535
17/95 [====>.........................] - ETA: 1:14 - loss: 39.9083 - mae: 4.9014
18/95 [====>.........................] - ETA: 1:12 - loss: 41.0297 - mae: 4.9688
19/95 [=====>........................] - ETA: 1:11 - loss: 41.6401 - mae: 5.0058
20/95 [=====>........................] - ETA: 1:10 - loss: 41.7900 - mae: 5.0021
21/95 [=====>........................] - ETA: 1:09 - loss: 41.5352 - mae: 5.0022
22/95 [=====>........................] - ETA: 1:08 - loss: 41.6305 - mae: 5.0026
23/95 [======>.......................] - ETA: 1:07 - loss: 41.4050 - mae: 4.9889
24/95 [======>.......................] - ETA: 1:06 - loss: 41.3400 - mae: 4.9772
25/95 [======>.......................] - ETA: 1:06 - loss: 41.4159 - mae: 4.9911
26/95 [=======>......................] - ETA: 1:05 - loss: 41.3161 - mae: 4.9758
27/95 [=======>......................] - ETA: 1:05 - loss: 41.5899 - mae: 4.9800
28/95 [=======>......................] - ETA: 1:04 - loss: 41.8916 - mae: 5.0112
29/95 [========>.....................] - ETA: 1:02 - loss: 41.9496 - mae: 5.0211
30/95 [========>.....................] - ETA: 1:01 - loss: 42.4464 - mae: 5.0497
31/95 [========>.....................] - ETA: 1:00 - loss: 42.2667 - mae: 5.0403
32/95 [=========>....................] - ETA: 59s - loss: 42.8201 - mae: 5.0685 
33/95 [=========>....................] - ETA: 58s - loss: 43.0104 - mae: 5.0759
34/95 [=========>....................] - ETA: 57s - loss: 42.5493 - mae: 5.0440
35/95 [==========>...................] - ETA: 56s - loss: 42.2793 - mae: 5.0305
36/95 [==========>...................] - ETA: 55s - loss: 42.6691 - mae: 5.0427
37/95 [==========>...................] - ETA: 54s - loss: 42.7116 - mae: 5.0444
38/95 [===========>..................] - ETA: 53s - loss: 42.7034 - mae: 5.0398
39/95 [===========>..................] - ETA: 53s - loss: 42.8732 - mae: 5.0439
40/95 [===========>..................] - ETA: 52s - loss: 43.0571 - mae: 5.0634
41/95 [===========>..................] - ETA: 51s - loss: 43.1912 - mae: 5.0638
42/95 [============>.................] - ETA: 50s - loss: 42.9635 - mae: 5.0519
43/95 [============>.................] - ETA: 49s - loss: 42.9587 - mae: 5.0564
44/95 [============>.................] - ETA: 48s - loss: 42.7749 - mae: 5.0410
45/95 [=============>................] - ETA: 47s - loss: 42.8748 - mae: 5.0464
46/95 [=============>................] - ETA: 46s - loss: 43.0480 - mae: 5.0616
47/95 [=============>................] - ETA: 45s - loss: 42.8959 - mae: 5.0546
48/95 [==============>...............] - ETA: 44s - loss: 42.8183 - mae: 5.0489
49/95 [==============>...............] - ETA: 43s - loss: 42.5705 - mae: 5.0321
50/95 [==============>...............] - ETA: 42s - loss: 42.4427 - mae: 5.0262
51/95 [===============>..............] - ETA: 41s - loss: 42.4249 - mae: 5.0275
52/95 [===============>..............] - ETA: 40s - loss: 42.6353 - mae: 5.0452
53/95 [===============>..............] - ETA: 39s - loss: 42.5016 - mae: 5.0413
54/95 [================>.............] - ETA: 38s - loss: 42.6429 - mae: 5.0506
55/95 [================>.............] - ETA: 38s - loss: 42.6556 - mae: 5.0488
56/95 [================>.............] - ETA: 37s - loss: 42.6536 - mae: 5.0508
57/95 [=================>............] - ETA: 36s - loss: 42.4067 - mae: 5.0351
58/95 [=================>............] - ETA: 35s - loss: 42.3227 - mae: 5.0328
59/95 [=================>............] - ETA: 34s - loss: 42.2013 - mae: 5.0239
60/95 [=================>............] - ETA: 33s - loss: 42.3086 - mae: 5.0268
61/95 [==================>...........] - ETA: 32s - loss: 42.0399 - mae: 5.0105
62/95 [==================>...........] - ETA: 31s - loss: 41.9941 - mae: 5.0085
63/95 [==================>...........] - ETA: 30s - loss: 41.9077 - mae: 5.0021
64/95 [===================>..........] - ETA: 29s - loss: 42.1420 - mae: 5.0074
65/95 [===================>..........] - ETA: 28s - loss: 42.2410 - mae: 5.0161
66/95 [===================>..........] - ETA: 27s - loss: 42.2921 - mae: 5.0201
67/95 [====================>.........] - ETA: 26s - loss: 42.1949 - mae: 5.0206
68/95 [====================>.........] - ETA: 25s - loss: 42.1368 - mae: 5.0157
69/95 [====================>.........] - ETA: 24s - loss: 42.1560 - mae: 5.0170
70/95 [=====================>........] - ETA: 23s - loss: 41.9659 - mae: 5.0065
71/95 [=====================>........] - ETA: 22s - loss: 41.9361 - mae: 5.0032
72/95 [=====================>........] - ETA: 21s - loss: 41.7041 - mae: 4.9877
73/95 [======================>.......] - ETA: 20s - loss: 41.8066 - mae: 4.9938
74/95 [======================>.......] - ETA: 19s - loss: 41.8356 - mae: 4.9928
75/95 [======================>.......] - ETA: 19s - loss: 41.6917 - mae: 4.9841
76/95 [=======================>......] - ETA: 18s - loss: 41.6129 - mae: 4.9767
77/95 [=======================>......] - ETA: 17s - loss: 41.6427 - mae: 4.9760
78/95 [=======================>......] - ETA: 16s - loss: 41.6713 - mae: 4.9797
79/95 [=======================>......] - ETA: 15s - loss: 41.7397 - mae: 4.9831
80/95 [========================>.....] - ETA: 14s - loss: 41.8032 - mae: 4.9860
81/95 [========================>.....] - ETA: 13s - loss: 41.8015 - mae: 4.9877
82/95 [========================>.....] - ETA: 12s - loss: 41.5904 - mae: 4.9758
83/95 [=========================>....] - ETA: 11s - loss: 41.4872 - mae: 4.9712
84/95 [=========================>....] - ETA: 10s - loss: 41.6063 - mae: 4.9784
85/95 [=========================>....] - ETA: 9s - loss: 41.5185 - mae: 4.9713 
86/95 [==========================>...] - ETA: 8s - loss: 41.4176 - mae: 4.9637
87/95 [==========================>...] - ETA: 7s - loss: 41.3645 - mae: 4.9596
88/95 [==========================>...] - ETA: 6s - loss: 41.4012 - mae: 4.9639
89/95 [===========================>..] - ETA: 5s - loss: 41.4623 - mae: 4.9676
90/95 [===========================>..] - ETA: 4s - loss: 41.5013 - mae: 4.9688
91/95 [===========================>..] - ETA: 3s - loss: 41.6245 - mae: 4.9722
92/95 [============================>.] - ETA: 2s - loss: 41.5862 - mae: 4.9717
93/95 [============================>.] - ETA: 1s - loss: 41.7688 - mae: 4.9783
94/95 [============================>.] - ETA: 0s - loss: 41.7054 - mae: 4.9722
95/95 [==============================] - 98s 1s/step - loss: 41.8713 - mae: 4.9800 - val_loss: 86.0388 - val_mae: 6.7765
Epoch 19/20

 1/95 [..............................] - ETA: 1:43 - loss: 33.3823 - mae: 4.5348
 2/95 [..............................] - ETA: 1:33 - loss: 38.6653 - mae: 4.8547
 3/95 [..............................] - ETA: 1:31 - loss: 36.2942 - mae: 4.7116
 4/95 [>.............................] - ETA: 1:28 - loss: 35.2375 - mae: 4.6151
 5/95 [>.............................] - ETA: 1:27 - loss: 34.0060 - mae: 4.4961
 6/95 [>.............................] - ETA: 1:25 - loss: 34.1386 - mae: 4.4531
 7/95 [=>............................] - ETA: 1:24 - loss: 34.2262 - mae: 4.5147
 8/95 [=>............................] - ETA: 1:23 - loss: 33.3187 - mae: 4.4636
 9/95 [=>............................] - ETA: 1:21 - loss: 34.2525 - mae: 4.5570
10/95 [==>...........................] - ETA: 1:20 - loss: 34.3167 - mae: 4.5942
11/95 [==>...........................] - ETA: 1:19 - loss: 34.8009 - mae: 4.6262
12/95 [==>...........................] - ETA: 1:18 - loss: 35.8544 - mae: 4.6776
13/95 [===>..........................] - ETA: 1:18 - loss: 36.5533 - mae: 4.7030
14/95 [===>..........................] - ETA: 1:17 - loss: 37.8445 - mae: 4.7772
15/95 [===>..........................] - ETA: 1:16 - loss: 38.5496 - mae: 4.8314
16/95 [====>.........................] - ETA: 1:15 - loss: 38.9022 - mae: 4.8364
17/95 [====>.........................] - ETA: 1:15 - loss: 39.6070 - mae: 4.8811
18/95 [====>.........................] - ETA: 1:14 - loss: 39.3793 - mae: 4.8770
19/95 [=====>........................] - ETA: 1:13 - loss: 39.1390 - mae: 4.8685
20/95 [=====>........................] - ETA: 1:12 - loss: 39.0140 - mae: 4.8665
21/95 [=====>........................] - ETA: 1:12 - loss: 39.2860 - mae: 4.8922
22/95 [=====>........................] - ETA: 1:11 - loss: 39.6275 - mae: 4.9169
23/95 [======>.......................] - ETA: 1:10 - loss: 39.3493 - mae: 4.8983
24/95 [======>.......................] - ETA: 1:08 - loss: 39.4782 - mae: 4.9029
25/95 [======>.......................] - ETA: 1:07 - loss: 39.6926 - mae: 4.9045
26/95 [=======>......................] - ETA: 1:06 - loss: 39.5713 - mae: 4.8967
27/95 [=======>......................] - ETA: 1:06 - loss: 39.6375 - mae: 4.8826
28/95 [=======>......................] - ETA: 1:04 - loss: 39.4455 - mae: 4.8702
29/95 [========>.....................] - ETA: 1:03 - loss: 38.9939 - mae: 4.8355
30/95 [========>.....................] - ETA: 1:02 - loss: 38.7829 - mae: 4.8221
31/95 [========>.....................] - ETA: 1:01 - loss: 38.6161 - mae: 4.8127
32/95 [=========>....................] - ETA: 1:00 - loss: 38.3186 - mae: 4.7912
33/95 [=========>....................] - ETA: 59s - loss: 38.5252 - mae: 4.8046 
34/95 [=========>....................] - ETA: 58s - loss: 38.5318 - mae: 4.7950
35/95 [==========>...................] - ETA: 57s - loss: 38.5124 - mae: 4.7914
36/95 [==========>...................] - ETA: 56s - loss: 38.7487 - mae: 4.7977
37/95 [==========>...................] - ETA: 55s - loss: 38.5753 - mae: 4.7933
38/95 [===========>..................] - ETA: 54s - loss: 38.6741 - mae: 4.8011
39/95 [===========>..................] - ETA: 53s - loss: 38.5136 - mae: 4.7933
40/95 [===========>..................] - ETA: 52s - loss: 38.8163 - mae: 4.8094
41/95 [===========>..................] - ETA: 51s - loss: 38.8744 - mae: 4.8134
42/95 [============>.................] - ETA: 50s - loss: 38.6938 - mae: 4.7947
43/95 [============>.................] - ETA: 49s - loss: 38.6954 - mae: 4.7962
44/95 [============>.................] - ETA: 48s - loss: 38.5793 - mae: 4.7886
45/95 [=============>................] - ETA: 48s - loss: 38.4750 - mae: 4.7767
46/95 [=============>................] - ETA: 46s - loss: 38.3402 - mae: 4.7643
47/95 [=============>................] - ETA: 46s - loss: 38.3495 - mae: 4.7660
48/95 [==============>...............] - ETA: 45s - loss: 38.4671 - mae: 4.7749
49/95 [==============>...............] - ETA: 44s - loss: 38.6689 - mae: 4.7938
50/95 [==============>...............] - ETA: 43s - loss: 38.4341 - mae: 4.7818
51/95 [===============>..............] - ETA: 42s - loss: 38.5564 - mae: 4.7919
52/95 [===============>..............] - ETA: 41s - loss: 38.6606 - mae: 4.8047
53/95 [===============>..............] - ETA: 40s - loss: 38.4360 - mae: 4.7919
54/95 [================>.............] - ETA: 39s - loss: 38.4363 - mae: 4.7914
55/95 [================>.............] - ETA: 38s - loss: 38.5465 - mae: 4.7958
56/95 [================>.............] - ETA: 37s - loss: 38.5647 - mae: 4.7946
57/95 [=================>............] - ETA: 36s - loss: 38.3725 - mae: 4.7818
58/95 [=================>............] - ETA: 35s - loss: 38.2654 - mae: 4.7750
59/95 [=================>............] - ETA: 34s - loss: 38.2186 - mae: 4.7753
60/95 [=================>............] - ETA: 33s - loss: 38.1958 - mae: 4.7696
61/95 [==================>...........] - ETA: 32s - loss: 38.2521 - mae: 4.7720
62/95 [==================>...........] - ETA: 31s - loss: 38.1277 - mae: 4.7631
63/95 [==================>...........] - ETA: 30s - loss: 38.1370 - mae: 4.7649
64/95 [===================>..........] - ETA: 29s - loss: 38.1593 - mae: 4.7658
65/95 [===================>..........] - ETA: 28s - loss: 38.1420 - mae: 4.7601
66/95 [===================>..........] - ETA: 27s - loss: 38.0282 - mae: 4.7538
67/95 [====================>.........] - ETA: 26s - loss: 37.9014 - mae: 4.7446
68/95 [====================>.........] - ETA: 25s - loss: 38.2849 - mae: 4.7715
69/95 [====================>.........] - ETA: 24s - loss: 38.1948 - mae: 4.7675
70/95 [=====================>........] - ETA: 23s - loss: 38.3708 - mae: 4.7759
71/95 [=====================>........] - ETA: 22s - loss: 38.3980 - mae: 4.7806
72/95 [=====================>........] - ETA: 21s - loss: 38.7212 - mae: 4.7935
73/95 [======================>.......] - ETA: 20s - loss: 38.6989 - mae: 4.7935
74/95 [======================>.......] - ETA: 19s - loss: 38.5768 - mae: 4.7869
75/95 [======================>.......] - ETA: 19s - loss: 38.5108 - mae: 4.7825
76/95 [=======================>......] - ETA: 18s - loss: 38.5715 - mae: 4.7865
77/95 [=======================>......] - ETA: 17s - loss: 38.6959 - mae: 4.7967
78/95 [=======================>......] - ETA: 16s - loss: 38.8146 - mae: 4.8102
79/95 [=======================>......] - ETA: 15s - loss: 38.8466 - mae: 4.8145
80/95 [========================>.....] - ETA: 14s - loss: 38.7229 - mae: 4.8050
81/95 [========================>.....] - ETA: 13s - loss: 38.6896 - mae: 4.8034
82/95 [========================>.....] - ETA: 12s - loss: 38.6312 - mae: 4.7994
83/95 [=========================>....] - ETA: 11s - loss: 38.8803 - mae: 4.8089
84/95 [=========================>....] - ETA: 10s - loss: 38.9772 - mae: 4.8122
85/95 [=========================>....] - ETA: 9s - loss: 38.9756 - mae: 4.8103 
86/95 [==========================>...] - ETA: 8s - loss: 38.9108 - mae: 4.8020
87/95 [==========================>...] - ETA: 7s - loss: 38.8345 - mae: 4.7995
88/95 [==========================>...] - ETA: 6s - loss: 38.6857 - mae: 4.7896
89/95 [===========================>..] - ETA: 5s - loss: 38.7255 - mae: 4.7882
90/95 [===========================>..] - ETA: 4s - loss: 38.7252 - mae: 4.7907
91/95 [===========================>..] - ETA: 3s - loss: 38.7724 - mae: 4.7932
92/95 [============================>.] - ETA: 2s - loss: 38.7866 - mae: 4.7928
93/95 [============================>.] - ETA: 1s - loss: 38.8374 - mae: 4.7922
94/95 [============================>.] - ETA: 0s - loss: 38.9407 - mae: 4.8021
95/95 [==============================] - 98s 1s/step - loss: 38.9456 - mae: 4.8032 - val_loss: 75.0638 - val_mae: 6.3499
Epoch 20/20

 1/95 [..............................] - ETA: 1:45 - loss: 16.8500 - mae: 3.3278
 2/95 [..............................] - ETA: 1:35 - loss: 25.9251 - mae: 4.0093
 3/95 [..............................] - ETA: 1:30 - loss: 27.4518 - mae: 4.1356
 4/95 [>.............................] - ETA: 1:30 - loss: 30.9064 - mae: 4.3583
 5/95 [>.............................] - ETA: 1:28 - loss: 31.5650 - mae: 4.3805
 6/95 [>.............................] - ETA: 1:26 - loss: 33.1105 - mae: 4.4839
 7/95 [=>............................] - ETA: 1:24 - loss: 33.9846 - mae: 4.5270
 8/95 [=>............................] - ETA: 1:23 - loss: 33.1569 - mae: 4.4943
 9/95 [=>............................] - ETA: 1:22 - loss: 32.6183 - mae: 4.4637
10/95 [==>...........................] - ETA: 1:22 - loss: 33.4370 - mae: 4.5503
11/95 [==>...........................] - ETA: 1:21 - loss: 33.7130 - mae: 4.5778
12/95 [==>...........................] - ETA: 1:20 - loss: 32.8012 - mae: 4.5258
13/95 [===>..........................] - ETA: 1:20 - loss: 33.6879 - mae: 4.5271
14/95 [===>..........................] - ETA: 1:19 - loss: 35.4421 - mae: 4.5676
15/95 [===>..........................] - ETA: 1:18 - loss: 34.8176 - mae: 4.5271
16/95 [====>.........................] - ETA: 1:17 - loss: 34.6398 - mae: 4.5357
17/95 [====>.........................] - ETA: 1:15 - loss: 34.8480 - mae: 4.5403
18/95 [====>.........................] - ETA: 1:14 - loss: 34.8405 - mae: 4.5426
19/95 [=====>........................] - ETA: 1:13 - loss: 35.6730 - mae: 4.5808
20/95 [=====>........................] - ETA: 1:12 - loss: 35.8395 - mae: 4.5912
21/95 [=====>........................] - ETA: 1:11 - loss: 35.8680 - mae: 4.5739
22/95 [=====>........................] - ETA: 1:10 - loss: 35.9015 - mae: 4.5714
23/95 [======>.......................] - ETA: 1:09 - loss: 35.4085 - mae: 4.5379
24/95 [======>.......................] - ETA: 1:08 - loss: 35.2750 - mae: 4.5342
25/95 [======>.......................] - ETA: 1:07 - loss: 35.2348 - mae: 4.5354
26/95 [=======>......................] - ETA: 1:06 - loss: 34.7912 - mae: 4.5097
27/95 [=======>......................] - ETA: 1:05 - loss: 34.8965 - mae: 4.5211
28/95 [=======>......................] - ETA: 1:04 - loss: 35.0009 - mae: 4.5227
29/95 [========>.....................] - ETA: 1:03 - loss: 35.6098 - mae: 4.5605
30/95 [========>.....................] - ETA: 1:02 - loss: 35.4720 - mae: 4.5581
31/95 [========>.....................] - ETA: 1:01 - loss: 35.2039 - mae: 4.5332
32/95 [=========>....................] - ETA: 1:00 - loss: 34.9859 - mae: 4.5225
33/95 [=========>....................] - ETA: 59s - loss: 34.9290 - mae: 4.5263 
34/95 [=========>....................] - ETA: 58s - loss: 34.6635 - mae: 4.5055
35/95 [==========>...................] - ETA: 57s - loss: 34.8825 - mae: 4.5131
36/95 [==========>...................] - ETA: 56s - loss: 34.7664 - mae: 4.5042
37/95 [==========>...................] - ETA: 55s - loss: 34.6137 - mae: 4.4959
38/95 [===========>..................] - ETA: 55s - loss: 34.5399 - mae: 4.4876
39/95 [===========>..................] - ETA: 54s - loss: 34.6056 - mae: 4.5024
40/95 [===========>..................] - ETA: 53s - loss: 34.8345 - mae: 4.5094
41/95 [===========>..................] - ETA: 52s - loss: 35.0284 - mae: 4.5190
42/95 [============>.................] - ETA: 51s - loss: 35.2879 - mae: 4.5267
43/95 [============>.................] - ETA: 50s - loss: 35.1983 - mae: 4.5171
44/95 [============>.................] - ETA: 49s - loss: 35.1836 - mae: 4.5222
45/95 [=============>................] - ETA: 48s - loss: 35.3448 - mae: 4.5326
46/95 [=============>................] - ETA: 47s - loss: 35.3854 - mae: 4.5362
47/95 [=============>................] - ETA: 46s - loss: 35.3287 - mae: 4.5398
48/95 [==============>...............] - ETA: 45s - loss: 35.3821 - mae: 4.5433
49/95 [==============>...............] - ETA: 44s - loss: 35.5840 - mae: 4.5579
50/95 [==============>...............] - ETA: 43s - loss: 35.5626 - mae: 4.5606
51/95 [===============>..............] - ETA: 42s - loss: 35.7554 - mae: 4.5734
52/95 [===============>..............] - ETA: 41s - loss: 35.8202 - mae: 4.5782
53/95 [===============>..............] - ETA: 40s - loss: 36.1425 - mae: 4.6038
54/95 [================>.............] - ETA: 39s - loss: 35.8645 - mae: 4.5870
55/95 [================>.............] - ETA: 38s - loss: 35.9672 - mae: 4.5988
56/95 [================>.............] - ETA: 37s - loss: 35.8460 - mae: 4.5887
57/95 [=================>............] - ETA: 36s - loss: 36.1407 - mae: 4.6014
58/95 [=================>............] - ETA: 35s - loss: 36.1511 - mae: 4.6024
59/95 [=================>............] - ETA: 34s - loss: 36.0810 - mae: 4.6041
60/95 [=================>............] - ETA: 33s - loss: 35.9210 - mae: 4.5948
61/95 [==================>...........] - ETA: 32s - loss: 35.8542 - mae: 4.5911
62/95 [==================>...........] - ETA: 31s - loss: 35.9420 - mae: 4.5998
63/95 [==================>...........] - ETA: 30s - loss: 35.9014 - mae: 4.5939
64/95 [===================>..........] - ETA: 29s - loss: 35.7928 - mae: 4.5911
65/95 [===================>..........] - ETA: 28s - loss: 35.8293 - mae: 4.5941
66/95 [===================>..........] - ETA: 27s - loss: 35.7051 - mae: 4.5889
67/95 [====================>.........] - ETA: 26s - loss: 35.6951 - mae: 4.5907
68/95 [====================>.........] - ETA: 25s - loss: 35.8122 - mae: 4.5971
69/95 [====================>.........] - ETA: 24s - loss: 35.7497 - mae: 4.5960
70/95 [=====================>........] - ETA: 23s - loss: 35.6273 - mae: 4.5895
71/95 [=====================>........] - ETA: 22s - loss: 35.4646 - mae: 4.5804
72/95 [=====================>........] - ETA: 21s - loss: 35.5220 - mae: 4.5808
73/95 [======================>.......] - ETA: 20s - loss: 35.3968 - mae: 4.5699
74/95 [======================>.......] - ETA: 20s - loss: 35.4365 - mae: 4.5709
75/95 [======================>.......] - ETA: 19s - loss: 35.5773 - mae: 4.5804
76/95 [=======================>......] - ETA: 18s - loss: 35.6788 - mae: 4.5881
77/95 [=======================>......] - ETA: 17s - loss: 35.7338 - mae: 4.5937
78/95 [=======================>......] - ETA: 16s - loss: 35.8410 - mae: 4.6037
79/95 [=======================>......] - ETA: 15s - loss: 35.6823 - mae: 4.5941
80/95 [========================>.....] - ETA: 14s - loss: 35.6865 - mae: 4.5888
81/95 [========================>.....] - ETA: 13s - loss: 35.6690 - mae: 4.5884
82/95 [========================>.....] - ETA: 12s - loss: 35.6065 - mae: 4.5818
83/95 [=========================>....] - ETA: 11s - loss: 35.8741 - mae: 4.5996
84/95 [=========================>....] - ETA: 10s - loss: 35.9030 - mae: 4.6044
85/95 [=========================>....] - ETA: 9s - loss: 36.0451 - mae: 4.6099 
86/95 [==========================>...] - ETA: 8s - loss: 36.0184 - mae: 4.6086
87/95 [==========================>...] - ETA: 7s - loss: 36.3605 - mae: 4.6326
88/95 [==========================>...] - ETA: 6s - loss: 36.2676 - mae: 4.6252
89/95 [===========================>..] - ETA: 5s - loss: 36.2899 - mae: 4.6245
90/95 [===========================>..] - ETA: 4s - loss: 36.2301 - mae: 4.6192
91/95 [===========================>..] - ETA: 3s - loss: 36.3700 - mae: 4.6328
92/95 [============================>.] - ETA: 2s - loss: 36.3132 - mae: 4.6292
93/95 [============================>.] - ETA: 1s - loss: 36.4695 - mae: 4.6370
94/95 [============================>.] - ETA: 0s - loss: 36.5062 - mae: 4.6373
95/95 [==============================] - 99s 1s/step - loss: 36.4372 - mae: 4.6342 - val_loss: 77.1930 - val_mae: 6.4156
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
24/24 - 8s - loss: 77.1930 - mae: 6.4156


Test MAE: 6.4156

## Conclusions

Based on the way this model was trained, and the metrics that were calculated, it seems that Computer Vision could help Good Seed determine the age of its customers to an extent. However, given the penalities for selling alcohol to those who are underage, and the fact that there is still a margin for error, I would recommend perhaps using the model to determine the age at which to request identification for a customer. Lots of businesses ID under 40 years old, and many of the images in the dataset fell in that category. The model could identify those customers and then a human could ask for identification from there.